<a href="https://colab.research.google.com/github/cardstdani/practica-java/blob/main/Pr%C3%A1cticaJAVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Generación de diccionario**

In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant
!sudo apt install myspell-es
!pip install unidecode

In [ ]:
import enchant
d = enchant.Dict("es")
print(d.check("habra"))
print(d.suggest("habra"))

In [ ]:
import itertools
import unidecode

d = enchant.Dict("es")
def checkString(n):
  return True if d.check(n) else False

l = [chr(i).upper() for i in range(97, 123)]+[i.upper() for i in ["á", "é", "í", "ó", "ú", "ü"]]
s = ["".join(i) for i in itertools.product(l, repeat=5)]

print(l)
print(s[:10], len(s))

Comprobadas +33m combinaciones de palabras en 5 minutos

In [ ]:
out = []
for i in s:
  if checkString(i):
    out.append(unidecode.unidecode(i))
    
out = sorted(list(set(out)))

In [ ]:
print(out[:20], len(out))

In [ ]:
textFile = open("Output.txt", "w")
textFile.write(" ".join(out))
textFile.close()

**Generar combinaciones**

In [ ]:
c = [list(i) for i in itertools.product([0,1,2], repeat=5)]

print(str(c).replace("'", "\"").replace("[", "{").replace("]", "}"), len(c))

In [ ]:
#Regex example
import re
pattern = re.compile("^(?=[^T]*$)(?!.{1}[N])(?=.*N)(?=.*A).*$")
print([i for i in d if re.match(pattern, i)])

**Generar todos los diccionarios necesarios**

In [ ]:
import json
import os
import re
import math
import random
import requests
import concurrent
from concurrent.futures import ProcessPoolExecutor


def generatePattern(entrada, word):
    pattern = ""
    procesed = {}

    for j in range(len(entrada)):
        letra = word[j]
        if letra not in procesed:
            condition = [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] == 2]
            if entrada[j] == 0:
                if condition == []: procesed[letra] = 0
            else:
                procesed[letra] = 0
            pattern += [f"(?=[^{letra}]*$)" if condition == [] else f"(?!.{{{j}}}{letra})",
                        f"(?!.{{{j}}}{letra})(?=.*{letra})",
                        f"(?=.{{{j}}}{letra})" + "".join(f"(?!.{{{i}}}{letra})" for i in [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] in [0, 1]])][entrada[j]]
    return f"^{pattern}.*$"


def scoreWord(word, d):
    combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101",
                    "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210",
                    "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012",
                    "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121",
                    "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000",
                    "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102",
                    "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211",
                    "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020",
                    "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122",
                    "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001",
                    "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110",
                    "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212",
                    "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021",
                    "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200",
                    "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002",
                    "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111",
                    "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220",
                    "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022",
                    "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201",
                    "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010",
                    "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112",
                    "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221",
                    "22222"]
    finalScore = 0

    for c in combinations:
        pattern = generatePattern([int(i) for i in c], word)
        p = 0
        for i in d.keys(): p += 1 if re.match(pattern, i) else 0
        p /= len(d)
        finalScore += p * math.log(p, 2) if p > 0 else 0
    # print(f"{word}:{finalScore}")
    return finalScore


def paralelDict(item, d):
    return {i: scoreWord(i, d) for i in item}


def updateDict(d, pattern):
    d = {k: 0 for (k, v) in d.items() if re.match(pattern, k)}

    n = os.cpu_count()
    chunkSize = math.ceil(len(d) / n)
    out = {}
    with ProcessPoolExecutor(n) as executor:
        futures = [executor.submit(paralelDict, list(d.keys())[chunkSize * i:chunkSize * (i + 1)], d) for i in range(n)]
        for future in concurrent.futures.as_completed(futures):
            out.update(future.result())
        executor.shutdown()
    return out


if __name__ == '__main__':
    combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101",
                    "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210",
                    "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012",
                    "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121",
                    "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000",
                    "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102",
                    "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211",
                    "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020",
                    "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122",
                    "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001",
                    "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110",
                    "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212",
                    "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021",
                    "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200",
                    "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002",
                    "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111",
                    "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220",
                    "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022",
                    "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201",
                    "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010",
                    "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112",
                    "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221",
                    "22222"]    

    d = json.loads(requests.get("https://media.githubusercontent.com/media/cardstdani/practica-java/main/Data/DictScoreData.txt").text)
    def precalcDict(d, i):
        maxWord = max(d, key=d.get)
        for c in combinations:
            pattern = generatePattern([int(i) for i in c], maxWord)
            if len({k: 0 for (k, v) in d.items() if re.match(pattern, k)}) > 100:
                tmpDict = updateDict(d, pattern)
                with open(f"Dict{i}-{c}.txt", 'w') as file:
                    file.write(json.dumps(tmpDict))
                precalcDict(tmpDict, i+1)

    precalcDict(d, 1)


In [ ]:
#@title Getting the dictionary ready
import numpy as np

l = 350 #@param {type:"integer"}
s = "ABABA ABACO ABADA ABADO ABAIS ABAJA ABAJE ABAJO ABANA ABANE ABANO ABAOS ABARA ABARE ABASE ABATA ABATE ABATI ABATO ABECE ABEIS ABEJA ABETO ABIAR ABOBA ABOBE ABOBO ABOCA ABOCO ABOGA ABOGO ABOLA ABOLE ABOLI ABOLO ABONA ABONE ABONO ABOYA ABOYE ABOYO ABRAN ABRAS ABREN ABRES ABRIA ABRID ABRIL ABRIO ABRIR ABRIS ABUCE ABUJE ABUSA ABUSE ABUSO ABUZA ABUZO ACABA ACABE ACABO ACAMA ACAME ACAMO ACANA ACARO ACASO ACATA ACATE ACATO ACEBO ACECE ACEDA ACEDE ACEDO ACEPA ACEPE ACEPO ACERA ACERE ACERO ACEZA ACEZO ACIDA ACIDO ACIMO ACLES ACODA ACODE ACODO ACOGE ACOGI ACOJA ACOJO ACORO ACOSA ACOSE ACOSO ACOTA ACOTE ACOTO ACREA ACREE ACREI ACREO ACRES ACTAS ACTEA ACTOR ACTOS ACTUA ACTUE ACTUO ACUDA ACUDE ACUDI ACUDO ACUEA ACUEO ACULA ACULE ACULO ACUNA ACUNE ACUNO ACURE ACUSA ACUSE ACUSO ACUTI ADAMA ADAME ADAMO ADIOS ADOBA ADOBE ADOBO ADORA ADORE ADORO ADOSA ADOSE ADOSO ADRAD ADRAN ADRAR ADRAS ADREN ADRES ADUAR ADUCE ADUJE ADUJO ADULA ADULE ADULO ADVEN AEDOS AEREA AEREO AETAS AFAMA AFAME AFAMO AFANA AFANE AFANO AFATA AFATE AFATO AFEAD AFEAN AFEAR AFEAS AFEEN AFEES AFIJA AFIJO AFILA AFILE AFILO AFINA AFINE AFINO AFLUI AFORA AFORE AFORO AFROS AFTAS AGAPE AGATA AGITA AGITE AGITO AGORA AGORE AGORO AGOTA AGOTE AGOTO AGRAZ AGRIA AGRIE AGRIO AGUAD AGUAN AGUAR AGUAS AGUCE AGUDA AGUDO AGUEN AGUES AGUJA AGUZA AGUZO AHAJA AHAJE AHAJO AHIJA AHIJE AHIJO AHILA AHILE AHILO AHITA AHITE AHITO AHOGA AHOGO AHORA AHOYA AHOYE AHOYO AHUMA AHUME AHUMO AHUSA AHUSE AHUSO AINAS AIRAD AIRAR AIREA AIREE AIREO AIRES AIRON AISLA AISLE AISLO AJABA AJADA AJADO AJAIS AJALA AJALE AJALO AJAME AJAOS AJARA AJARE AJASE AJATE AJEAD AJEAN AJEAR AJEAS AJEEN AJEES AJEIS AJELA AJELE AJELO AJEME AJENA AJENO AJEOS AJERA AJERO AJESE AJETE AJORA AJORE AJORO AJOTE AJUAR AJUMA AJUME AJUMO ALABA ALABE ALABO ALADA ALADO ALAFA ALAGA ALAGO ALAJU ALAMA ALAMO ALANO ALAUI ALAZO ALBAR ALBAS ALBIN ALBOR ALBOS ALBUM ALBUR ALCEA ALCEN ALCES ALCOR ALDEA ALEAD ALEAN ALEAR ALEAS ALECE ALEDA ALEEN ALEES ALEGA ALEGO ALEJA ALEJE ALEJO ALELA ALELE ALELI ALELO ALERO ALETA ALETO ALEYA ALEZO ALFAD ALFAN ALFAR ALFAS ALFEN ALFES ALFIL ALFIZ ALFOZ ALGAR ALGAS ALGOL ALGUN ALIAD ALIAN ALIAR ALIAS ALICA ALIEN ALIES ALIFA ALIGA ALIGO ALIJA ALIJE ALIJO ALIMO ALISA ALISE ALISO ALJEZ ALLAS ALMAS ALMOS ALMUD ALOCA ALOCO ALOJA ALOJE ALOJO ALOMA ALOME ALOMO ALOSA ALOTA ALOTE ALOTO ALOYA ALPES ALTAR ALTAS ALTEA ALTEE ALTEO ALTOS ALUDA ALUDE ALUDI ALUDO ALVEA ALVEO ALZAD ALZAN ALZAR ALZAS AMABA AMADA AMADO AMAGA AMAGO AMAIS AMALA AMALE AMALO AMAME AMAOS AMARA AMARE AMARO AMASA AMASE AMASO AMATE AMBAR AMBAS AMBLA AMBLE AMBLO AMBON AMBOS AMEBA AMEIS AMELA AMELE AMELO AMEME AMENA AMENO AMEOS AMERA AMERE AMERO AMESE AMIAS AMIBA AMIBO AMIDA AMIGA AMIGO AMINO AMIRI AMITO AMOLA AMOLE AMOLO AMOMO AMPON AMPRA AMPRE AMPRO AMULA AMULE AMULO AMURA AMURE AMURO AMUSO ANACO ANADE ANAFE ANATA ANCAS ANCHA ANCHE ANCHO ANCLA ANCLE ANCLO ANCON ANDAD ANDAN ANDAR ANDAS ANDEN ANDES ANDON ANEAS ANEGA ANEGO ANEJA ANEJE ANEJO ANETO ANEXA ANEXE ANEXO ANGEL ANGLA ANGLO ANIDA ANIDE ANIDO ANIMA ANIME ANIMO ANION ANITO ANODO ANONA ANOTA ANOTE ANOTO ANSAR ANSAS ANSIA ANSIE ANSIO ANTES ANTIA ANTRO ANUAL ANUDA ANUDE ANUDO ANULA ANULE ANULO ANUOS ANURA ANURO AOJAD AOJAN AOJAR AOJAS AOJEN AOJES AORTA AOVAD AOVAN AOVAR AOVAS AOVEN AOVES APAGA APAGO APARA APARE APARO APEAD APEAN APEAR APEAS APEEN APEES APEGA APEGO APELA APELE APELO APENA APENE APENO APEOS APERA APERE APERO APICE APILA APILE APILO APIOS APNEA APOCA APOCO APODA APODE APODO APOYA APOYE APOYO APROA APROE APROO APTAS APTOS APURA APURE APURO AQUEL AQUEO ARABA ARABE ARABO ARADA ARADO ARAIS ARALA ARALE ARALO ARAME ARANA ARAOS ARARA ARARE ARASE ARATE ARAZA ARBOL ARCAD ARCAN ARCAR ARCAS ARCEA ARCEN ARCES ARCHA ARCON ARCOS ARDAN ARDAS ARDED ARDEN ARDER ARDES ARDIA ARDID ARDIO ARDOR ARDUA ARDUO AREAS AREIS ARELA ARELE ARELO AREME ARENA ARENE ARENO ARESE ARETE ARGEL ARGON ARGOS ARGOT ARGUI ARIAS ARICA ARICO ARIDA ARIDO ARIES ARIOS ARMAD ARMAN ARMAR ARMAS ARMEN ARMES ARNES AROMA AROME AROMO ARPAD ARPAN ARPAR ARPAS ARPEN ARPEO ARPES ARPIA ARPON ARQUE ARRAS ARREA ARREE ARREO ARRES ARRIA ARRIE ARRIO ARROZ ARRUA ARRUE ARRUO ARTES ARZON ASABA ASADA ASADO ASAIS ASALA ASALE ASALO ASAME ASAOS ASARA ASARE ASASE ASATE ASCII ASCOS ASCUA ASEAD ASEAN ASEAR ASEAS ASEDA ASEDE ASEDO ASEEN ASEES ASEIS ASELA ASELE ASELO ASEME ASEOS ASESA ASESE ASESO ASIAN ASIAS ASIDA ASIDO ASILA ASILE ASILO ASINA ASIOS ASIRA ASIRE ASMAS ASNAL ASNOS ASOLA ASOLE ASOLO ASOMA ASOME ASOMO ASONE ASONO ASPAS ASPID ASTAN ASTAS ASTED ASTEN ASTER ASTES ASTIA ASTIO ASTRO ASTUR ASUMA ASUME ASUMI ASUMO ASURA ASURE ASURO ATABA ATABE ATACA ATACO ATADA ATADO ATAIS ATAJA ATAJE ATAJO ATALA ATALE ATALO ATAME ATAOS ATARA ATARE ATASE ATATE ATAUD ATEAS ATECE ATEIS ATELA ATELE ATELO ATEME ATEOS ATESE ATETA ATETE ATETO ATEZA ATEZO ATIBA ATIBE ATIBO ATICA ATICE ATICO ATINA ATINE ATINO ATIZA ATIZO ATLAS ATOAD ATOAN ATOAR ATOAS ATOEN ATOES ATOMO ATONA ATONO ATORA ATORE ATORO ATRAE ATRAS ATRIL ATRIO ATROZ ATUFA ATUFE ATUFO ATUSA ATUSE ATUSO ATUVE ATUVO AUDAZ AUDIO AUGES AUGUR AULAS AULLA AULLE AULLO AUNAD AUNAN AUNAR AUNAS AUNEN AUNES AUPAD AUPAN AUPAR AUPAS AUPEN AUPES AURAS AUREA AUREO AUTOR AUTOS AVALA AVALE AVALO AVARA AVARO AVECE AVENA AVENE AVENO AVEZA AVEZO AVIAD AVIAN AVIAR AVIAS AVICA AVIDA AVIDO AVIEN AVIES AVINE AVINO AVION AVISA AVISE AVISO AVIVA AVIVE AVIVO AVOCA AVOCO AVUGO AXIAL AXILA AYATE AYUDA AYUDE AYUDO AYUNA AYUNE AYUNO AZADA AZALA AZARA AZARE AZARO AZIMA AZIMO AZOAD AZOAN AZOAR AZOAS AZOCA AZOCO AZOEN AZOES AZOGA AZOGO AZOLE AZOLO AZORA AZORE AZORO AZOTA AZOTE AZOTO AZUAS AZUCE AZULA AZULE AZULO AZUZA AZUZO BABAS BABEA BABEE BABEL BABEO BABIA BABLE BABOR BACAS BACHE BACIA BACIN BACON BADAL BADAN BADEA BADEN BAFLE BAGAD BAGAN BAGAR BAGAS BAGOS BAGRE BAGUE BAHIA BAIDA BAILA BAILE BAILO BAJAD BAJAN BAJAR BAJAS BAJEL BAJEN BAJES BAJON BAJOS BALAD BALAN BALAR BALAS BALDA BALDE BALDO BALEA BALEN BALEO BALES BALIN BALON BALSA BALSO BALTA BALTO BAMBA BAMBU BANAL BANCA BANCE BANCO BANDA BANDO BANTU BANZO BAQUE BARBA BARBE BARBO BARCA BARCO BARDA BARDE BARDO BARES BARIA BARIO BARON BARRA BARRE BARRI BARRO BARZA BASAD BASAL BASAN BASAR BASAS BASCA BASEN BASES BASTA BASTE BASTO BATAN BATAS BATEA BATEE BATEL BATEN BATEO BATES BATIA BATID BATIN BATIO BATIR BATIS BATOS BAUZA BAYAL BAYAS BAYON BAYOS BAZAR BAZAS BAZOS BEATA BEATO BEBAN BEBAS BEBED BEBEN BEBER BEBES BEBIA BEBIO BECAD BECAN BECAR BECAS BEDEL BEFAD BEFAN BEFAR BEFAS BEFEN BEFES BEIGE BELDA BELDE BELDO BELEN BELEZ BELFA BELFO BELGA BELIO BELLA BELLO BEMOL BEODA BEODO BEQUE BERRO BERTA BERZA BESAD BESAN BESAR BESAS BESEN BESES BESOS BETAS BETUN BEZAR BICHA BICHO BICIS BIDES BIDON BIELA BIGAS BIJAS BILAO BILIS BIMBA BINAD BINAN BINAR BINAS BINEN BINES BINGO BINZA BIRLA BIRLE BIRLO BISEL BISOS BITER BIZCA BIZCO BLEDO BLOCA BLOCO BLUES BLUSA BOATO BOBAS BOBEA BOBEE BOBEO BOBOS BOCAL BOCAS BOCEA BOCEE BOCEO BOCHA BOCHE BOCHO BOCIN BOCIO BOCON BODAS BODON BOFES BOGAD BOGAN BOGAR BOGAS BOGUE BOHIO BOINA BOIRA BOJES BOLAR BOLAS BOLEA BOLEE BOLEO BOLES BOLIN BOLLA BOLLE BOLLO BOLOS BOLSA BOLSO BOMBA BOMBE BOMBO BONGA BONGO BONOS BONZO BORAX BORDA BORDE BORDO BOREA BOREO BORLA BORNE BOROS BORRA BORRE BORRO BOSTA BOTAD BOTAN BOTAR BOTAS BOTEN BOTES BOTIN BOTON BOXEA BOXEE BOXEO BOXER BOYAD BOYAL BOYAN BOYAR BOYAS BOYEN BOYES BOZAL BOZOS BRACO BRAGA BRAMA BRAME BRAMO BRASA BRAVA BRAVO BRAZA BRAZO BREAD BREAN BREAR BREAS BRECA BREEN BREES BREGA BREGO BRETE BREVA BREVE BREZO BRIDA BRIOS BRISA BRIZA BROCA BROCE BROMA BROME BROMO BROTA BROTE BROTO BROZA BROZO BRUGO BRUJA BRUJO BRUMA BRUTA BRUTO BUBAS BUBON BUCAL BUCEA BUCEE BUCEN BUCEO BUCES BUCHE BUCLE BUCOS BUDAS BUDIN BUENA BUENO BUFAD BUFAN BUFAR BUFAS BUFEN BUFES BUFON BUFOS BUHOS BUJES BUJIA BULAS BULBO BULLA BULLE BULLI BULLO BULOS BULTO BUQUE BURDA BURDO BUREO BURGA BURGO BURIL BURLA BURLE BURLO BUROS BURRA BURRO BUSCA BUSCO BUSES BUSTO BUYOS BUZAD BUZAN BUZAR BUZAS BUZON BUZOS BYTES CABAL CABAS CABED CABEN CABER CABES CABIA CABLE CABOS CABRA CABRE CACAO CACAS CACEA CACEE CACEN CACEO CACES CACHA CACHE CACHO CACOS CACTO CAEIS CAEME CAEOS CAERA CAERE CAETE CAFES CAFRE CAGAD CAGAN CAGAR CAGAS CAGON CAGUE CAIAN CAIAS CAICO CAIDA CAIDO CAIGA CAIGO CAJAS CAJIN CAJON CALAD CALAN CALAO CALAR CALAS CALCA CALCE CALCO CALDA CALDO CALEN CALES CALIZ CALLA CALLE CALLO CALMA CALME CALMO CALON CALOR CALOS CALVA CALVE CALVO CALZA CALZO CAMAS CAMBA CAMBO CAMON CAMPA CAMPE CAMPO CANAL CANAS CANDA CANDE CANDO CANEA CANEE CANEO CANES CANEY CANJE CANOA CANON CANOS CANSA CANSE CANSO CANTA CANTE CANTO CAOBA CAPAD CAPAN CAPAR CAPAS CAPAZ CAPEA CAPEE CAPEN CAPEO CAPES CAPIN CAPON CAPTA CAPTE CAPTO CAQUI CARAS CARAY CARCA CARDA CARDE CARDO CAREA CAREE CAREO CAREY CARGA CARGO CARIA CARIE CARIO CARIZ CARLA CARLO CARNE CAROS CARPA CARPO CARRO CARTA CASAD CASAL CASAN CASAR CASAS CASCA CASCO CASEA CASEN CASEO CASES CASIO CASON CASOS CASPA CASTA CASTO CATAD CATAN CATAR CATAS CATEA CATEE CATEN CATEO CATES CATIN CATON CATOS CATRE CAUCE CAUSA CAUSE CAUSO CAUTA CAUTO CAVAD CAVAN CAVAR CAVAS CAVEA CAVEN CAVES CAYOS CAZAD CAZAN CAZAR CAZAS CAZON CAZUZ CCITT CEBAD CEBAN CEBAR CEBAS CEBEN CEBES CEBON CEBOS CEBRA CECEA CECEE CECEO CEDAN CEDAS CEDED CEDEN CEDER CEDES CEDIA CEDIO CEDRO CEGAD CEGAR CEGUE CEJAD CEJAN CEJAR CEJAS CEJEN CEJES CEJOS CELAD CELAN CELAR CELAS CELDA CELEN CELES CELIA CELOS CELTA CENAD CENAN CENAR CENAS CENEN CENES CENIT CENSA CENSE CENSO CEPAS CEPOS CERAS CERCA CERCO CERDA CERDO CEREA CEREO CERIO CERNI CERON CEROS CERRE CERRO CESAD CESAN CESAR CESAS CESEN CESES CESIO CESTA CESTO CETRO CEUTI CHACO CHAFA CHAFE CHAFO CHALA CHALE CHALO CHAPA CHAPE CHAPO CHATA CHATO CHECA CHECO CHEFS CHEPA CHICA CHICO CHILE CHINA CHINE CHINO CHIPS CHIST CHITO CHIVA CHIVE CHIVO CHOCA CHOCO CHOLA CHOPO CHOTA CHOTO CHOVA CHOZA CHOZO CHUFA CHULA CHULO CHUPA CHUPE CHUPO CHUTA CHUTE CHUTO CHUZA CHUZO CIABA CIADA CIADO CIAIS CIARA CIARE CIASE CICLA CICLE CICLO CIDES CIDRA CIDRO CIEGA CIEGO CIEIS CIELO CIENO CIFRA CIFRE CIFRO CILIO CIMAS CINCA CINCO CINES CINTA CINTE CINTO CIRCE CIRCO CIRIO CIRRO CISCA CISCO CISMA CISME CISMO CISNE CITAD CITAN CITAR CITAS CITEN CITES CITOS CIVIL CLAMA CLAME CLAMO CLARA CLARO CLASE CLAVA CLAVE CLAVO CLERO CLIMA CLISE CLOCO CLONA CLONE CLONO CLORA CLORE CLORO COBAS COBOL COBRA COBRE COBRO COCAD COCAN COCAR COCAS COCEA COCED COCEE COCEO COCER COCES COCHE COCIA COCIO COCOS CODEA CODEE CODEO CODOS COFAS COFIA COFRE COGED COGEN COGER COGES COGIA COGIO COGON COITO COJAN COJAS COJEA COJEE COJEO COJIN COJOS COLAD COLAR COLAS COLEA COLEE COLEO COLES COLGO COLIN COLMA COLME COLMO COLON COLOR COLZA COMAN COMAS COMBA COMBE COMBO COMED COMEN COMER COMES COMIA COMIC COMIO COMUN CONDE CONGA CONGO CONOS CONTE CONTO COPAD COPAN COPAR COPAS COPEA COPEE COPEN COPEO COPES COPIA COPIE COPIO COPLA COPON COPOS COQUE CORAL CORAN CORDA COREA COREE COREO CORNO COROS CORRA CORRE CORRI CORRO CORSA CORSE CORSO CORTA CORTE CORTO CORVA CORVO CORZA CORZO COSAN COSAS COSCA COSCO COSED COSEN COSER COSES COSIA COSIO COSOS COSTA COSTE COSTO COTAD COTAN COTAR COTAS COTEN COTES COTON COTOS COXAL COXIS CRASA CRASO CREAD CREAN CREAR CREAS CRECE CRECI CREDO CREED CREEN CREER CREES CREIA CREMA CRETA CREYO CRIAD CRIAN CRIAR CRIAS CRIBA CRIBE CRIBO CRIEN CRIES CRIOS CROAD CROAN CROAR CROAS CROEN CROES CROMA CROME CROMO CRONO CRUCE CRUDA CRUDO CRUEL CRUJA CRUJE CRUJI CRUJO CRUOR CRUZA CRUZO CUAJA CUAJE CUAJO CUBAS CUBIL CUBOS CUBRA CUBRE CUBRI CUBRO CUCAD CUCAN CUCAR CUCAS CUCOS CUECE CUELA CUELE CUELO CUERA CUERO CUEVA CUEZA CUEZO CUIDA CUIDE CUIDO CUITA CULAR CULEA CULEE CULEO CULON CULOS CULPA CULPE CULPO CULTA CULTO CUNAD CUNAN CUNAR CUNAS CUNDA CUNDE CUNDI CUNDO CUNEA CUNEE CUNEN CUNEO CUNES CUOTA CUPES CUPLE CUPON CUPOS CUQUE CURAD CURAN CURAR CURAS CURDA CURDO CUREN CURES CURIA CURIO CURRA CURRE CURRO CURSA CURSE CURSI CURSO CURTA CURTE CURTI CURTO CURVA CURVE CURVO CUTER CUTIS CUTOS CUTRE CUYAS CUYOS CUZCO DABAN DABAS DABLE DACAS DACHA DACIA DACIO DADAS DADLA DADLE DADLO DADME DADOR DADOS DAGAS DALAS DALES DALIA DALLE DALOS DAMAS DAMOS DANCE DANDI DANDO DANES DANOS DANTE DANZA DANZO DARAN DARAS DARDO DARIA DARLA DARLE DARLO DARME DAROS DARSE DARTE DATAD DATAN DATAR DATAS DATEN DATES DATIL DATOS DAVID DEBAN DEBAS DEBED DEBEN DEBER DEBES DEBIA DEBIL DEBIO DEBUT DECAE DECAI DECIA DECID DECIR DECIS DEDAL DEDIL DEDOS DEJAD DEJAN DEJAR DEJAS DEJEN DEJES DELAS DELES DELGA DELIA DELOS DELTA DEMAS DEMOS DENLA DENLE DENLO DENME DENOS DENSA DENSE DENSO DENTA DENTE DENTO DEPON DERBI DESDA DESDE DESDO DESEA DESEE DESEO DESGA DESOI DETEN DEUDA DEUDO DEVEN DIADA DIADO DIANA DICAZ DICEN DICES DICHA DICHO DICTA DICTE DICTO DIEGO DIERA DIERE DIESE DIETA DIETE DIETO DIGAN DIGAS DIGNA DIGNE DIGNO DIJES DILAS DILES DILOS DILUI DIMES DIMOS DINAR DINOS DIODO DIOSA DIQUE DIRAN DIRAS DIRIA DISCO DISON DISTA DISTE DISTO DITAS DIUCA DIVAN DIVAS DIVOS DOBLA DOBLE DOBLO DOCAS DOCES DOCIL DOCTA DOCTO DOGAL DOGAS DOGMA DOGOS DOGRE DOLAD DOLAR DOLED DOLER DOLIA DOLIO DOLOR DOLOS DOMAD DOMAN DOMAR DOMAS DOMBO DOMEN DOMES DOMOS DONAD DONAN DONAR DONAS DONDE DONEN DONES DOPAD DOPAN DOPAR DOPAS DOPEN DOPES DORAD DORAL DORAN DORAR DORAS DOREN DORES DORIA DORIO DORMI DORNA DORSO DOSEL DOSES DOSIS DOTAD DOTAL DOTAN DOTAR DOTAS DOTEN DOTES DRABA DRAGA DRAGO DRAMA DRENA DRENE DRENO DRIAS DRINO DRIZA DROGA DROGO DROPE DRUPA DRUSA DRUSO DUBAS DUBIO DUCAL DUCES DUCHA DUCHE DUCHO DUDAD DUDAN DUDAR DUDAS DUDEN DUDES DUELA DUELE DUELO DUETO DUJOS DULAR DULAS DULCE DULIA DUMAS DUNAS DUNDA DUNDO DUPLA DUPLO DUQUE DURAD DURAN DURAR DURAS DUREN DURES DUROS EBANO EBRIA EBRIO ECHAD ECHAN ECHAR ECHAS ECHEN ECHES ECUAS ECUOS EDEMA EDILA EDITA EDITE EDITO EDRAD EDRAN EDRAR EDRAS EDREN EDRES EDUCA EDUCO EFEBO EFETA EFLUI EGIDA EIRAS EJIDO EJION ELATO ELCHE ELEGI ELEPE ELEVA ELEVE ELEVO ELFOS ELIDA ELIDE ELIDI ELIDO ELIGE ELIJA ELIJE ELIJO ELITE ELLAS ELLES ELLOS ELOTE ELUDA ELUDE ELUDI ELUDO EMANA EMANE EMANO EMITA EMITE EMITI EMITO EMULA EMULE EMULO ENANA ENANO ENCIA ENDES ENEAL ENEAS ENEJA ENEJE ENEJO ENEMA ENERO ENOJA ENOJE ENOJO ENRIA ENRIE ENRIO ENSAY ENTES ENTRA ENTRE ENTRO ENVES ENVIA ENVIE ENVIO ENZAS EOLIA EOLIO EONES EPATA EPATE EPATO EPICA EPICO EPOCA EPOXI EPROM EQUIS ERABA ERADA ERADO ERAIS ERAJE ERALA ERARA ERARE ERASE ERBIO EREBO EREIS ERGIO ERGOS ERGUI ERIAL ERIAS ERICE ERIGE ERIGI ERIJA ERIJO ERINA ERIOS ERIZA ERIZO EROGA EROGO ERRAD ERRAN ERRAR ERRAS ERREN ERRES ERROR ERUTA ERUTE ERUTO ESCOA ESMUI ESNOB ESPIA ESPIE ESPIN ESPIO ESQUI ESTAD ESTAN ESTAR ESTAS ESTEN ESTER ESTES ESTIO ESTOS ESTOY ESTRO ETANO ETAPA ETICA ETICO ETILO ETIMO ETNIA EUROS EVADA EVADE EVADI EVADO EVITA EVITE EVITO EVOCA EVOCO EXIGE EXIGI EXIJA EXIJO EXILA EXILE EXILO EXIMA EXIME EXIMI EXIMO EXITO EXODO EXORA EXORE EXORO EXPIA EXPIE EXPIO EXPON EXTRA EXUDA EXUDE EXUDO FABLA FACAS FACES FACHA FACIL FACON FADOS FAENA FAENE FAENO FAGOT FAJAD FAJAN FAJAR FAJAS FAJEN FAJES FAJIN FAJON FAJOS FALAZ FALCA FALDA FALLA FALLE FALLO FALOS FALSA FALSO FALTA FALTE FALTO FALUA FAMAS FANAL FANES FANGO FARDA FARDE FARDO FAROL FAROS FARRA FARSA FASES FASTA FASTO FATAL FATOS FATUA FATUO FAUNA FAUNO FAVOR FAXES FEBLE FECAL FECHA FECHE FECHO FELIX FELIZ FELON FELPA FELPE FELPO FEMUR FENDA FENIX FENOL FERAZ FERIA FERIE FERIO FEROE FEROZ FERRA FERRE FERRO FERRY FETAL FETCH FETEN FETOS FEUCA FEUCO FEUDA FEUDE FEUDO FIABA FIADA FIADO FIAIS FIALA FIALE FIALO FIAME FIAOS FIARA FIARE FIASE FIATE FIBRA FICHA FICHE FICHO FICUS FIDEO FIEIS FIELA FIELE FIELO FIEME FIERA FIERO FIESE FIGLE FIGON FIJAD FIJAN FIJAR FIJAS FIJEN FIJES FIJOS FILAD FILAN FILAR FILAS FILEN FILES FILFA FILIA FILIE FILIO FILMA FILME FILMO FILON FILOS FINAD FINAL FINAN FINAR FINAS FINCA FINCO FINEN FINES FINGE FINGI FINJA FINJO FINOS FINTA FINTE FINTO FIQUE FIRMA FIRME FIRMO FISCO FISGA FISGO FLACA FLACO FLAMA FLASH FLATO FLECO FLEJE FLEMA FLETA FLETE FLETO FLEXO FLOJA FLOJO FLORA FLORE FLORO FLOTA FLOTE FLOTO FLUIA FLUID FLUIR FLUIS FLUJO FLUOR FLUYA FLUYE FLUYO FOBIA FOCAL FOCAS FOCHA FOCOS FOFAS FOFOS FOGON FOJAS FOLIA FOLIE FOLIO FOLLA FOLLE FOLLO FONDA FONDO FONJE FORAL FORCE FORJA FORJE FORJO FORMA FORME FORMO FOROS FORRA FORRE FORRO FORZO FOSAD FOSAN FOSAR FOSAS FOSCA FOSCO FOSEN FOSES FOSIL FOSOS FOTON FOTOS FRAGA FRASE FRECE FREGO FREIA FREID FREIR FREIS FRENA FRENE FRENO FREON FREOS FRESA FRESE FRESO FREZA FREZO FRIAN FRIAS FRIEN FRIES FRIOS FRISA FRISE FRISO FRITA FRITE FRITO FROTA FROTE FROTO FRUTA FRUTE FRUTO FUCOS FUEGO FUERA FUERE FUERO FUESE FUGAD FUGAN FUGAR FUGAS FUGAZ FUGUE FUINA FULAR FULGE FULGI FULJA FULJO FUMAD FUMAN FUMAR FUMAS FUMEN FUMES FUNDA FUNDE FUNDI FUNDO FURIA FUROR FUSAS FUSCA FUSIL FUSOR FUSTA FUSTE FUTIL FUTRE GABAN GACEL GACHA GACHI GACHO GAFAD GAFAN GAFAR GAFAS GAFEN GAFES GAFOS GAGAS GAGOS GAITA GAJES GAJOS GALAN GALAS GALES GALGA GALGO GALIO GALLO GALON GALOS GAMAS GAMBA GAMMA GAMON GAMOS GANAD GANAN GANAR GANAS GANEN GANES GANGA GANSA GANSO GANTE GARBA GARBO GARFA GARLA GARRA GARUA GARUE GARUO GARZA GARZO GASAS GASEA GASEE GASEO GASES GASON GASTA GASTE GASTO GATAS GATEA GATEE GATEO GATOS GAUSS GAVIA GAYAS GAYOS GAZAS GBYTE GELES GEMAS GEMIA GEMID GEMIR GEMIS GENES GENIA GENIO GENTE GESTA GESTE GESTO GFLOP GHCES GIBAD GIBAN GIBAR GIBAS GIBEN GIBES GIGAS GILES GIMAN GIMAS GIMEN GIMES GIMIO GIRAD GIRAN GIRAR GIRAS GIREN GIRES GIROS GISES GLASE GLEBA GLIAL GLIAS GLIFO GLOBO GLOSA GLOSE GLOSO GNEIS GNOMO GOBIO GOCEN GOCES GOCHA GOCHO GODAS GODOS GOFIO GOFRA GOFRE GOFRO GOLAS GOLEA GOLEE GOLEO GOLES GOLFA GOLFO GOLPE GOMAS GOMER GORDA GORDO GORJA GORRA GORRO GOTAS GOTEA GOTEE GOTEO GOTON GOZAD GOZAN GOZAR GOZAS GOZNE GOZOS GRABA GRABE GRABO GRADA GRADE GRADO GRAFO GRAJA GRAJO GRAMA GRAMO GRANA GRANE GRANO GRAOS GRAPA GRAPE GRAPO GRASA GRASO GRATA GRATO GRAVA GRAVE GRAVO GREBA GRECA GRECO GREDA GREEN GRELO GRIAL GRIFA GRIFO GRILL GRIMA GRIPE GRISU GRITA GRITE GRITO GROSO GRUAS GRUMO GRUPA GRUPO GRUTA GUACA GUACO GUAJE GUAMA GUANO GUAPA GUAPO GUARA GUARO GUASA GUASO GUATA GUATE GUAYA GUAYE GUAYO GUBIA GUETO GUIAD GUIAN GUIAR GUIAS GUIDO GUIEN GUIES GUIJA GUIJO GUION GUIPA GUIPE GUIPO GUIRA GUIRI GUISA GUISE GUISO GUITA GUITE GUITO GULAG GULAS GUMIA GURDA GURDO GUROS GUSTA GUSTE GUSTO GUZLA HABAR HABAS HABER HABIA HABIL HABIZ HABLA HABLE HABLO HABON HABRA HABRE HABUS HACAN HACAS HACED HACEN HACER HACES HACHA HACHE HACHO HACIA HADAS HADOS HAFIZ HAGAN HAGAS HALAD HALAN HALAR HALAS HALDA HALEN HALES HALLA HALLE HALLO HALON HALOS HAMEZ HAMPA HANZO HARAN HARAS HARBA HARBE HARBO HARCA HAREM HAREN HARIA HARON HARPA HARRE HARTA HARTE HARTO HASTA HATEA HATEE HATEO HATOS HAUTE HAVAR HAVOS HAYAL HAYAN HAYAS HAYOS HAZAS HAZLA HAZLE HAZLO HAZME HAZTE HEBEN HEBRA HECES HECHA HECHO HECTO HEDED HEDER HEDIA HEDIO HEDOR HELAD HELAR HELEA HELEE HELEO HELIO HELOR HEMOS HENAL HENAR HENDA HENDE HENDI HENDO HENIL HENOS HERBE HERBO HERIA HERID HERIL HERIR HERIS HERMA HEROE HERPE HERRE HERRO HERVI HESPA HESPE HESPI HESPO HETEA HETEO HIATO HIDRA HIDRO HIEDA HIEDE HIEDO HIELA HIELE HIELO HIENA HIERA HIERE HIERO HIGAS HIGOS HIJAS HIJOS HILAD HILAN HILAR HILAS HILEN HILES HILOS HIMEN HIMNO HINCA HINCO HINDU HINES HIPAD HIPAN HIPAR HIPAS HIPEN HIPER HIPES HIPOS HIRCO HIRIO HIRMA HIRME HIRMO HISCA HISPA HISPE HISPI HISPO HITAD HITAN HITAR HITAS HITEN HITES HITON HITOS HOBOS HOCEN HOCES HOGAR HOJAS HOJEA HOJEE HOJEO HOLAN HOLAS HOLCO HOLEA HOLEE HOLEO HOLGO HOLLE HOLLO HONDA HONDO HONGO HONOR HONRA HONRE HONRO HOPAS HOPEA HOPEE HOPEO HOPOS HOQUE HORAS HORCA HORCO HORDA HORMA HORNO HORRA HORRE HORRO HOSCA HOSCO HOSPA HOTEL HOTOS HOVES HOYAS HOYOS HOZAD HOZAN HOZAR HOZAS HUACA HUACO HUCHA HUCHO HUECA HUECO HUELA HUELE HUELO HUERA HUERO HUESA HUESO HUEVA HUEVE HUEVO HUIAN HUIAS HUICH HUIDA HUIDO HUIFA HUIRA HUIRE HUIRO HULES HULLA HUMAD HUMAN HUMAR HUMAS HUMEA HUMEE HUMEN HUMEO HUMES HUMOR HUMOS HUMUS HUNAS HUNDA HUNDE HUNDI HUNDO HUNOS HUPES HURAS HURGA HURGO HURON HURRA HURTA HURTE HURTO HUSAR HUSMA HUSMO HUSOS HUTAS HUTIA HUYAN HUYAS HUYEN HUYES IBAIS IBERA IBERO IBICE ICACO ICEIS ICELA ICELE ICELO ICEME ICESE ICHAL ICHOS ICONO ICTUS IDEAD IDEAL IDEAN IDEAR IDEAS IDEEN IDEES IDEOS IDOLA IDOLO IGNEA IGNEO IGUAL IJADA IJUJU ILEAS ILEON ILEOS ILESA ILESO ILOTA ILUDA ILUDE ILUDI ILUDO ILUSA ILUSO IMADA IMANA IMANE IMANO IMBUI IMELA IMITA IMITE IMITO IMPAR IMPIA IMPIO IMPLA IMPON INANE INCAS INCOA INCOE INCOO INDAS INDIA INDIO INDOS INFLA INFLE INFLO INGLE INPUT INSTA INSTE INSTO INTER INTUI IONES IOTAS IRANI IREIS IRENE IRIAN IRIAS IRISA IRISE IRISO IRNOS ISAAC ISBAS ISLAM ISLAS ISTMO ITALA ITALO ITERA ITERE ITERO ITRIA ITRIO IZABA IZADA IZADO IZAIS IZALA IZALE IZALO IZAME IZAOS IZARA IZARE IZASE IZATE IZOTE JABAS JABON JACAS JACOS JACTA JACTE JACTO JADEA JADEE JADEO JADES JAECE JAEZA JAEZO JAIBA JAIMA JAIME JALAD JALAN JALAR JALAS JALDA JALDE JALDO JALEA JALEE JALEN JALEO JALES JALMA JALON JAMAD JAMAN JAMAR JAMAS JAMBA JAMEN JAMES JAMON JAPON JAQUE JARAL JARAS JAROS JARRA JARRE JARRO JASAD JASAN JASAR JASAS JASEN JASES JASPE JATAS JATEA JATEO JATIB JATOS JAUDA JAUDO JAUJA JAULA JAUTA JAUTO JAVAS JAVOS JAYAN JEBES JEDAD JEDAN JEDAR JEDAS JEDEN JEDES JEFAS JEFES JEITO JEJEN JELIZ JEMAL JEMES JEQUE JERAS JERBO JEREZ JERGA JERPA JESUS JETAD JETAN JETAR JETAS JETEN JETES JETON JETOS JIBIA JIFAS JIFIA JIGAS JIJAS JIJEA JIJEE JIJEO JIMIA JIMIO JIRAS JIRON JISCA JITAD JITAN JITAR JITAS JITEN JITES JOBOS JOCOS JODAN JODAS JODED JODEN JODER JODES JODIA JODIO JOFOR JONDO JONIA JONIO JOPAD JOPAN JOPAR JOPAS JOPEN JOPES JOPOS JORAS JORCO JORFE JORGE JORRA JORRE JORRO JOSAS JOTAS JOTES JOULE JOVEN JOYAS JOYON JOYOS JUBAS JUBON JUBOS JUDAS JUDEA JUDIA JUDIO JUDIT JUEGA JUEGO JUERA JUEZA JUGAD JUGAR JUGOS JUGUE JUJEO JULIA JULIO JULOS JUMAS JUNCO JUNIO JUNTA JUNTE JUNTO JUNZA JURAD JURAN JURAR JURAS JUREL JUREN JURES JUROS JUSTA JUSTE JUSTO JUTAS JUTIA JUVIA JUZGA JUZGO KANES KAPPA KARMA KARST KBITS KBYTE KEFIR KHCES KILOS KIRIE KIVIS KIWIS KOPEK KURDA KURDO LABIA LABIL LABIO LABOR LABRA LABRE LABRO LACAD LACAN LACAR LACAS LACEA LACEE LACEN LACEO LACES LACHA LACIA LACIO LACON LACRA LACRE LACRO LACTA LACTE LACTO LADAS LADEA LADEE LADEO LADON LADOS LADRA LADRE LADRO LAGAR LAGOS LAICA LAICO LAJAS LAMAN LAMAS LAMED LAMEN LAMER LAMES LAMIA LAMIN LAMIO LAMPA LAMPE LAMPO LANAR LANAS LANCE LANDA LANDO LANGA LANIA LANIO LANZA LANZO LAPAS LAPIZ LAPON LAPOS LAPSO LAQUE LARDA LARDE LARDO LARES LARGA LARGO LARRA LARVA LASAS LASCA LASCO LASER LASOS LASTA LASTE LASTO LASUN LATAN LATAS LATAZ LATEN LATES LATEX LATIA LATID LATIN LATIO LATIR LATIS LATON LATOS LAUDA LAUDE LAUDO LAUNA LAURA LAURO LAVAD LAVAN LAVAR LAVAS LAVEN LAVES LAXAD LAXAN LAXAR LAXAS LAXEN LAXES LAXOS LAYAD LAYAN LAYAR LAYAS LAYEN LAYES LAZAD LAZAN LAZAR LAZAS LAZOS LEAIS LEALA LEALE LEALO LEAME LEASE LECHA LECHE LECHO LEDAS LEDOS LEEIS LEELA LEELE LEELO LEEME LEEOS LEERA LEERE LEETE LEGAD LEGAL LEGAN LEGAR LEGAS LEGON LEGOS LEGRA LEGRE LEGRO LEGUA LEGUE LEIAN LEIAS LEIDA LEIDO LEILA LEIMA LEJAS LEJIA LEJOS LELAS LELOS LEMAS LEMUR LENAS LENES LENTA LENTE LENTO LEONA LEPRA LERAS LERDA LERDO LESAS LESNA LESOS LESTE LETAL LETEA LETEO LETON LETRA LEUDA LEUDE LEUDO LEVAD LEVAN LEVAR LEVAS LEVEN LEVES LEYES LEZDA LEZNA LIABA LIADA LIADO LIAIS LIALA LIALE LIALO LIAME LIANA LIAOS LIARA LIARE LIASE LIATE LIAZA LIBAD LIBAN LIBAR LIBAS LIBEN LIBER LIBES LIBIA LIBIO LIBON LIBRA LIBRE LIBRO LICEO LICIA LICIO LICOR LICUA LICUE LICUO LIDER LIDES LIDIA LIDIE LIDIO LIEGA LIEGO LIEIS LIELA LIELE LIELO LIEME LIESE LIGAD LIGAN LIGAR LIGAS LIGIO LIGON LIGUE LIGUR LIJAD LIJAN LIJAR LIJAS LIJEN LIJES LILAC LILAO LILAS LIMAD LIMAN LIMAR LIMAS LIMBO LIMEN LIMES LIMON LIMOS LINAO LINAR LINCE LINDA LINDE LINDO LINEA LINEE LINEO LINFA LINIO LINON LINOS LINUX LIOSA LIOSO LIPES LIPIS LIRAS LIRIA LIRIO LIRON LISAS LISIA LISIE LISIO LISIS LISOS LISTA LISTE LISTO LITAD LITAN LITAR LITAS LITEN LITES LITIO LITIS LITRE LITRO LITUO LIVOR LIZAS LIZOS LLACA LLAGA LLAGO LLAMA LLAME LLAMO LLANA LLANO LLAPA LLAPE LLAPO LLAVE LLECA LLECO LLEGA LLEGO LLENA LLENE LLENO LLERA LLEVA LLEVE LLEVO LLORA LLORE LLORO LLOSA LLOVI LOABA LOADA LOADO LOAIS LOARA LOARE LOASE LOBAS LOBEA LOBEE LOBEO LOBOS LOCAL LOCAS LOCHA LOCHE LOCOS LOCRO LODON LODOS LOEIS LOGIA LOGIN LOGIS LOGOS LOGRA LOGRE LOGRO LOICA LOINA LOMAS LOMBA LOMEA LOMEE LOMEO LOMOS LONAS LONJA LORAS LORES LOROS LORZA LOSAD LOSAN LOSAR LOSAS LOSEN LOSES LOTAS LOTES LOTOS LOZAS LUCEN LUCES LUCHA LUCHE LUCHO LUCIA LUCID LUCIO LUCIR LUCIS LUCRA LUCRE LUCRO LUDAN LUDAS LUDEN LUDES LUDIA LUDID LUDIE LUDIO LUDIR LUDIS LUEGO LUGAR LUGRE LUIAN LUIAS LUIDA LUIDO LUIRA LUIRE LUISA LUJOS LUMAS LUMIA LUNAR LUNAS LUNES LUPAS LUPIA LUPUS LURTE LUSAS LUSOS LUTEA LUTEO LUTOS LUXAD LUXAN LUXAR LUXAS LUXEN LUXES LUYAN LUYAS LUYEN LUYES LUZCA LUZCO LYCRA MACAD MACAN MACAR MACAS MACEA MACEE MACEN MACEO MACES MACHA MACHE MACHI MACHO MACIA MACIS MACON MACOS MACRO MADOR MADRE MAFIA MAGAS MAGIA MAGIN MAGMA MAGNA MAGNO MAGOS MAGRA MAGRO MAHON MAIDO MAJAD MAJAL MAJAN MAJAR MAJAS MAJEN MAJES MAJOS MALAR MALAS MALEA MALEE MALEO MALES MALLA MALLE MALLO MALON MALOS MALTA MALVA MAMAD MAMAN MAMAR MAMAS MAMBI MAMBO MAMEN MAMES MAMEY MAMIA MAMON MAMUT MANAD MANAN MANAR MANAS MANCA MANCO MANDA MANDE MANDI MANDO MANEA MANEE MANEN MANEO MANES MANGA MANGO MANIA MANID MANIO MANIR MANIS MANOS MANSA MANSO MANTA MANTO MAORI MAPAS MAPEA MAPEE MAPEO MAQUE MAQUI MARCA MARCO MAREA MAREE MAREO MARES MARGA MARGO MARIO MARON MAROS MARRA MARRE MARRO MARSA MARSO MARTA MARTE MARZO MASAD MASAN MASAR MASAS MASCA MASCO MASEN MASES MASIA MASLO MASON MASTO MATAD MATAN MATAR MATAS MATEA MATEE MATEN MATEO MATES MATIZ MATON MATOS MAULA MAULE MAULO MAURA MAURE MAURO MAYAL MAYAS MAYEA MAYEE MAYEO MAYOR MAYOS MAZAD MAZAN MAZAR MAZAS MAZNA MAZNE MAZNO MAZOS MBITS MBYTE MEABA MEADA MEADO MEAIS MEAJA MEALA MEALE MEALO MEAME MEAOS MEARA MEARE MEASE MEATE MEATO MECED MECEN MECER MECES MECHA MECHE MECHO MECIA MECIO MECOS MEDAS MEDIA MEDID MEDIE MEDIO MEDIR MEDIS MEDOS MEDRA MEDRE MEDRO MEEIS MEELA MEELE MEELO MEEME MEESE MEGAS MEGOS MEIGA MEIGO MEJOR MELAD MELAR MELAS MELCA MELGA MELGO MELIS MELLA MELLE MELLO MELON MELSA MELVA MEMAS MEMEZ MEMOS MENAD MENAN MENAR MENAS MENDA MENEA MENEE MENEN MENEO MENES MENOR MENOS MENTA MENTE MENTI MENTO MENUS MEONA MERAD MERAN MERAR MERAS MERCA MERCO MEREN MERES MEREY MERGO MERLA MERLO MERMA MERME MERMO MEROS MESAD MESAN MESAR MESAS MESEN MESES MESON MESTA MESTO METAL METAN METAS METED METEN METER METES METIA METIO METRA METRO MEZAN MEZAS MFLOP MHCES MIABA MIADA MIADO MIAGA MIAGO MIAIS MIAJA MIARA MIARE MIASE MICAS MICER MICES MICHA MICHO MICOS MICRA MICRO MIDAN MIDAS MIDEN MIDES MIDIO MIEDO MIEIS MIELA MIELE MIELO MIERA MIGAD MIGAN MIGAR MIGAS MIGRA MIGRE MIGRO MIGUE MIJOS MILAN MILES MILLA MILLO MILPA MIMAD MIMAN MIMAR MIMAS MIMEN MIMES MIMOS MINAD MINAL MINAN MINAR MINAS MINEN MINES MINGA MINGO MINIA MINIE MINIO MINIX MINOS MINUE MIOMA MIOPE MIRAD MIRAN MIRAR MIRAS MIREN MIRES MIRIA MIRLA MIRLE MIRLO MIRON MIRRA MIRTO MIRZA MISAD MISAL MISAN MISAR MISAS MISEN MISES MISIA MISIL MISIO MISMA MISMO MISTA MISTE MISTO MITAD MITAN MITAS MITIN MITON MITOS MITRA MITRE MITRO MIXTA MIXTO MIZAS MIZOS MOARE MOCAD MOCAN MOCAR MOCAS MOCEA MOCEE MOCEO MOCHA MOCHE MOCHO MOCIL MOCOS MODAL MODAS MODEM MODIO MODOS MOFAD MOFAN MOFAR MOFAS MOFEN MOFES MOGOL MOGON MOHIN MOHOS MOHUR MOJAD MOJAN MOJAR MOJAS MOJEN MOJES MOJON MOJOS MOLAR MOLAS MOLDA MOLDE MOLDO MOLED MOLER MOLES MOLIA MOLIO MOLLA MOLLE MOLON MOLOS MOLSA MOLSO MOMEA MOMEE MOMEO MOMIA MOMIO MOMOS MONAS MONDA MONDE MONDO MONEA MONEE MONEO MONFI MONGO MONJA MONJE MONOS MONTA MONTE MONTO MOPAS MOQUE MORAD MORAL MORAN MORAR MORAS MORBO MORCA MORCO MORDI MOREA MOREN MOREO MORES MORFA MORGA MORIA MORID MORIR MORIS MORON MOROS MORRA MORRO MORSA MORSE MOSCA MOSCO MOSEN MOSTE MOSTO MOTAS MOTEA MOTEE MOTEL MOTEO MOTES MOTIF MOTIN MOTON MOTOR MOTOS MOVED MOVER MOVIA MOVIL MOVIO MOXAS MOXTE MOYAS MOYOS MOZAS MOZOS MUAIS MUARE MUCHA MUCHO MUDAD MUDAN MUDAR MUDAS MUDEN MUDES MUDEZ MUDOS MUECA MUEIS MUELA MUELE MUELO MUERA MUERE MUERO MUESO MUEVA MUEVE MUEVO MUFLA MUFLE MUFLI MUFLO MUFTI MUGAD MUGAN MUGAR MUGAS MUGEN MUGES MUGIA MUGID MUGIO MUGIR MUGIS MUGRE MUGUE MUIAN MUIAS MUIDA MUIDO MUJAN MUJAS MUJER MUJOL MULAR MULAS MULEO MULLA MULLE MULLI MULLO MULOS MULSA MULSO MULTA MULTE MULTO MUNDO MURAD MURAL MURAN MURAR MURAS MUREN MURES MURGA MURIA MURIO MUROS MURTA MUSAN MUSAS MUSCA MUSCO MUSEN MUSEO MUSES MUSGA MUSGO MUSIA MUSID MUSIO MUSIR MUSIS MUSLO MUTAD MUTAN MUTAR MUTAS MUTEN MUTES MUTIS MUTUA MUTUO NABAB NABAL NABAR NABAS NABLA NABOS NACAR NACED NACEN NACER NACES NACHA NACHO NACIA NACIO NACOS NADAD NADAN NADAR NADAS NADEN NADES NADIE NADIR NAFRA NAFRE NAFRO NAFTA NAGUA NAHUA NAIFE NAIPE NAIRE NAJAS NALCA NALGA NANAS NANCE NANEA NANEE NANEO NANSA NANSU NANTA NANTE NANTO NAPAS NAPEA NAPEO NAQUE NARDO NARES NARIZ NARRA NARRE NARRO NASAL NASAS NASON NASOS NATAL NATAS NATIA NATIO NATOS NAUTA NAVAL NAVAS NAVES NAVIO NAZCA NAZCO NAZIS NEBLI NECEA NECEE NECEO NECIA NECIO NEGAD NEGAR NEGRA NEGRO NEGUE NEGUS NEJAS NELDO NEMAS NEMEA NEMEO NEMES NENAS NENES NENIA NERON NESGA NESGO NETAS NETOS NEUMA NEURA NEVAD NEVAN NEVAR NEVAS NEVEN NEVES NEVUS NEXOS NIARA NIAZO NICHO NICLE NIDAL NIDIA NIDIO NIDOS NIEGA NIEGO NIELA NIELE NIELO NIETA NIETO NIEVE NIMBA NIMBE NIMBO NIMIA NIMIO NINFA NINFO NIOTO NIPAS NIPON NITOS NITRA NITRE NITRO NIVEA NIVEL NIVEO NOBEL NOBLE NOCAS NOCHE NOCLA NODOS NOGAL NOMAS NOMON NOMOS NONAS NONES NONIO NONOS NOPAL NORIA NORMA NORTE NOTAD NOTAN NOTAR NOTAS NOTEN NOTES NOTOS NOTRO NOVAD NOVAL NOVAN NOVAR NOVAS NOVEL NOVEN NOVES NOVIA NOVIO NOXAS NOYOS NUBES NUBIA NUBIL NUBIO NUBLA NUBLE NUBLO NUCAS NUCOS NUDAS NUDOS NUERA NUEVA NUEVE NUEVO NULAS NULOS NUMEN NUMOS NUNCA NURIA NUTRA NUTRE NUTRI NUTRO OASIS OBESA OBESO OBICE OBITO OBLEA OBOES OBOLO OBRAD OBRAN OBRAR OBRAS OBREN OBRES OBSTA OBSTE OBSTO OBTEN OBUES OBVIA OBVIE OBVIO OCASO OCELO OCENA OCHOS OCIAD OCIAN OCIAR OCIAS OCIEN OCIES OCIOS OCLES OCLUI OCRES OCTAL OCUJE OCUMO OCUPA OCUPE OCUPO ODEON ODIAD ODIAN ODIAR ODIAS ODIEN ODIES ODIOS ODRES OESTE OFITA OGROS OHMIO OIAIS OIBLE OIDAS OIDIO OIDLA OIDLE OIDLO OIDME OIDOR OIDOS OIGAN OIGAS OIMOS OIRAN OIRAS OIRIA OIRLA OIRLE OIRLO OIRME OIROS OIRSE OIRTE OISLO OISTE OJALA OJALE OJALO OJEAD OJEAN OJEAR OJEAS OJEEN OJEES OJEOS OJERA OJETE OJIVA OJOSA OJOSO OJOTA OLAIS OLAJE OLEAD OLEAN OLEAR OLEAS OLEEN OLEES OLEIS OLEOS OLERA OLERE OLIAN OLIAS OLIDA OLIDO OLIVA OLIVO OLLAR OLLAS OLMOS OMEGA OMEYA OMINA OMINE OMINO OMISA OMISO OMITA OMITE OMITI OMITO ONDAS ONDEA ONDEE ONDEO ONICE ONZAS OPACA OPACO OPALO OPERA OPERE OPERO OPINA OPINE OPINO OPIOS OPONE OPTAD OPTAN OPTAR OPTAS OPTEN OPTES OPUSE OPUSO ORABA ORADA ORADO ORAIS ORALA ORALE ORALO ORARA ORARE ORASE ORATE ORBES ORCAS ORCEN ORCES ORCOS ORDEN OREAD OREAN OREAR OREAS OREEN OREES OREIS OREJA ORELA ORELE ORELO ORFOS ORFRE ORGIA ORIBE ORINA ORINE ORINO ORION ORIVE ORLAD ORLAN ORLAR ORLAS ORLEN ORLES ORLOS ORNAR OROYA ORTOS ORUGA ORUJO ORZAD ORZAN ORZAR ORZAS OSABA OSADA OSADO OSAIS OSALA OSALE OSALO OSARA OSARE OSASE OSEAD OSEAN OSEAR OSEAS OSEEN OSEES OSEIS OSELA OSELE OSELO OSEOS OSERA OSERO OSMIO OSTIA OSTRA OSTRO OSUDA OSUDO OSUNA OSUNO OTEAD OTEAN OTEAR OTEAS OTEEN OTEES OTERO OTILA OTILE OTILO OTOBA OTRAS OTROS OVABA OVADA OVADO OVAIS OVALA OVALE OVALO OVARA OVARE OVASE OVEIS OVEJA OVERA OVERO OVIDO OVINA OVINO OVOLO OVULA OVULE OVULO OXIDA OXIDE OXIDO OXTES OYELA OYELE OYELO OYEME OYERA OYERE OYESE OYETE OZONA OZONO PABLO PACED PACEN PACER PACES PACIA PACIO PACON PACTA PACTE PACTO PADRE PAFIA PAFIO PAGAD PAGAN PAGAR PAGAS PAGOS PAGUE PAICO PAIRO PAJAR PAJAS PAJEA PAJEE PAJEO PAJES PAJIL PAJON PAJOS PALAS PALCO PALEA PALEE PALEO PALIA PALIE PALIO PALMA PALMO PALON PALOS PALPA PALPE PALPO PAMPA PANAL PANAS PANCO PANDA PANEL PANES PANJI PANSA PANZA PAPAD PAPAL PAPAN PAPAR PAPAS PAPEA PAPEE PAPEL PAPEN PAPEO PAPES PARAD PARAN PARAR PARAS PARCA PARCO PARDA PARDO PAREA PARED PAREE PAREN PAREO PARES PARGO PARIA PARID PARIO PARIR PARIS PARLA PARLE PARLO PARNE PARON PAROS PARRA PARTA PARTE PARTI PARTO PARVA PARVO PASAD PASAN PASAR PASAS PASEA PASEE PASEN PASEO PASES PASMA PASME PASMO PASOS PASTA PASTE PASTO PATAN PATAO PATAS PATEA PATEE PATEO PATER PATES PATIN PATIO PATON PATOS PAUSA PAUSE PAUSO PAUTA PAUTE PAUTO PAVAS PAVES PAVIA PAVON PAVOR PAVOS PAYAS PAYES PAYOS PAZAN PAZAS PAZOS PEAIS PEAJE PEANA PECAD PECAN PECAR PECAS PECES PECHA PECHE PECHO PECIO PEDAL PEDIA PEDID PEDIR PEDIS PEDOS PEDRO PEEIS PEEOS PEERA PEERE PEGAD PEGAN PEGAR PEGAS PEGON PEGUE PEIAN PEIAS PEIDA PEIDO PEINA PEINE PEINO PEJES PEJIN PELAD PELAN PELAR PELAS PELEA PELEE PELEN PELEO PELES PELLA PELLO PELMA PELON PELOS PELTA PENAD PENAL PENAN PENAR PENAS PENCA PENCO PENDA PENDE PENDI PENDO PENEN PENES PENOS PENSE PENSO PEPAS PEPES PEPON PEQUE PERAL PERAS PERCA PERDI PERLA PERLE PERLO PERNO PEROL PERRA PERRO PERSA PERTA PESAD PESAN PESAR PESAS PESCA PESCO PESEN PESES PESOS PESTE PETAD PETAN PETAR PETAS PETEN PETES PETOS PETRA PEZON PIABA PIADA PIADO PIAFA PIAFE PIAFO PIAIS PIANO PIARA PIARE PIASE PICAD PICAN PICAR PICAS PICHI PICIO PICON PICOR PICOS PIDAN PIDAS PIDEN PIDES PIDIO PIDON PIEIS PIEJO PIEZA PIFIA PIFIE PIFIO PILAR PILAS PILEO PILLA PILLE PILLO PILON PILOS PINAR PINCE PINES PINGA PINGO PINOS PINTA PINTE PINTO PINZA PINZO PIOJO PIOLA PIOLE PIOLO PIONA PIPAD PIPAN PIPAR PIPAS PIPEN PIPES PIPIA PIPIE PIPIO PIPOS PIQUE PIRAD PIRAN PIRAR PIRAS PIRCA PIRCO PIREN PIRES PIRON PIRRA PIRRE PIRRO PISAD PISAN PISAR PISAS PISCO PISEN PISES PISON PISOS PISPA PISTA PISTE PISTO PITAD PITAN PITAO PITAR PITAS PITEN PITES PITIA PITIO PITON PITOS PIULA PIULE PIULO PIUNE PIURE PIVOT PIXEL PIZCA PIZCO PIZZA PLACA PLACO PLAGA PLAGO PLANA PLANO PLATA PLATO PLAYA PLAZA PLAZO PLEBE PLECA PLEGO PLENA PLENO PLEON PLEXO PLICA PLISA PLISE PLISO PLOMA PLOME PLOMO PLUMA POBLE POBLO POBOS POBRE POCAS POCHA POCHO POCOS PODAD PODAN PODAR PODAS PODED PODEN PODER PODES PODIA PODIO PODON PODRA PODRE POEMA POETA POINO POLAR POLCA POLCO POLEA POLEN POLEO POLIN POLIO POLLA POLLO POLOS POLVO POMAR POMAS POMEZ POMOS POMPA POMPO PONCI PONED PONEN PONER PONES PONGA PONGO PONIA PONIS PONLA PONLE PONLO PONME PONTE PONTO POPAD POPAN POPAR POPAS POPEN POPES PORGA PORGO PORNO POROS PORRA PORRO PORTA PORTE PORTO POSAD POSAN POSAR POSAS POSCA POSEA POSEE POSEI POSEN POSEO POSES POSMA POSON POSOS POSTA POSTE POTAD POTAN POTAR POTAS POTEN POTES POTRA POTRO POYOS POZAL POZAS POZOS PRADO PRAVA PRAVO PRESA PRESO PREVE PREVI PRIMA PRIME PRIMO PRION PRIOR PRISA PRIVA PRIVE PRIVO PROAS PROBA PROBE PROBO PROIZ PROLE PRONO PRORA PROSA PRUNO PUABA PUADA PUADO PUAIS PUARA PUARE PUASE PUBER PUBES PUBIS PUCHO PUCIA PUDIO PUDOR PUDRA PUDRE PUDRI PUDRO PUEDA PUEDE PUEDO PUEIS PUFOS PUGAS PUGIL PUGNA PUGNE PUGNO PUJAD PUJAN PUJAR PUJAS PUJEN PUJES PUJOS PULAN PULAS PULEN PULES PULGA PULIA PULID PULIO PULIR PULIS PULLA PULPA PULPO PULSA PULSE PULSO PUMAS PUNAN PUNAS PUNCE PUNEN PUNES PUNGE PUNGI PUNIA PUNID PUNIO PUNIR PUNIS PUNJA PUNJO PUNTA PUNTE PUNTO PUNZA PUNZO PUPAS PURAS PUREA PUREE PUREO PURES PURGA PURGO PUROS PUTAS PUTEA PUTEE PUTEO PUTOS PUYAS PUYOS QUARK QUEDA QUEDE QUEDO QUEJA QUEJE QUEJO QUEMA QUEME QUEMI QUEMO QUENA QUEPA QUEPO QUERA QUESO QUIAS QUIEN QUIJO QUILA QUILO QUIMA QUIMO QUINA QUINO QUIOS QUISE QUISO QUITA QUITE QUITO QUIZA RABAS RABEA RABEE RABEL RABEO RABIA RABIE RABIO RABON RABOS RACHA RACHE RACHO RADAR RADAS RADIA RADIE RADIO RADIX RADON RAEIS RAERA RAERE RAFAS RAFEA RAFEE RAFEO RAFES RAFIA RAGUA RAHEZ RAIAN RAIAS RAIDA RAIDO RAIGA RAIGO RAIJO RAJAD RAJAN RAJAR RAJAS RAJEN RAJES RALAS RALEA RALEE RALEO RALLA RALLE RALLO RALLY RALOS RAMAL RAMAS RAMEA RAMEO RAMIO RAMON RAMOS RAMPA RAMPE RAMPO RANAS RANGO RANOS RAPAD RAPAN RAPAR RAPAS RAPAZ RAPEN RAPES RAPOS RAPTA RAPTE RAPTO RARAS RAREA RAREE RAREO RAROS RASAD RASAN RASAR RASAS RASCA RASCO RASEN RASES RASGA RASGO RASOS RASPA RASPE RASPO RATAS RATEA RATEE RATEO RATIO RATON RATOS RAUDA RAUDO RAULI RAUTA RAYAD RAYAN RAYAR RAYAS RAYEN RAYES RAYON RAYOS RAZAS RAZON REAMA REAME REAMO REARA REARE REARO REATA REATE REATO REBLA REBLE REBLO RECAE RECAI RECEN RECES RECIA RECIO RECRE RECTA RECTE RECTO RECUA REDES REDIL REDRO REGAD REGAR REGIA REGID REGIO REGIR REGIS REGLA REGLE REGLO REGUE REHAZ REHEN REHUI REIAN REIAS REIDA REIDO REINA REINE REINO REIOS REIRA REIRE REJAS REJON REJOS RELAX RELEA RELEE RELEI RELEJ RELEO RELES RELOJ RELSA RELSO REMAD REMAN REMAR REMAS REMEN REMES REMOS RENAL RENDA RENDI RENES RENGA RENGO RENOS RENTA RENTE RENTO REOJO REPON REPTA REPTE REPTO RESES RESOL RESTA RESTE RESTO RETAD RETAL RETAN RETAR RETAS RETEN RETES RETIN RETOR RETOS RETRO REUMA REUNA REUNE REUNI REUNO REUSA REUSE REUSO REVEN REVES REYES REZAD REZAN REZAR REZAS REZNO REZON REZOS RIADA RIAIS RIALA RIALE RIALO RIAME RIASE RICAS RICEN RICES RICIA RICIO RICOS RIEGA RIEGO RIELA RIELE RIELO RIEME RIERA RIERE RIESE RIETE RIFAD RIFAN RIFAR RIFAS RIFEN RIFES RIFLE RIGEN RIGES RIGIO RIGOR RIJAN RIJAS RIJOS RILAD RILAN RILAR RILAS RILEN RILES RIMAD RIMAN RIMAR RIMAS RIMEL RIMEN RIMES RINDA RINDE RINDO RINGA RINGO RIOJA RIPIA RIPIE RIPIO RISAS RISCA RISCO RISOS RISPA RISPO RITAS RITMA RITME RITMO RITOS RIVAL RIZAD RIZAN RIZAR RIZAS RIZON RIZOS ROAIS ROALA ROALE ROALO ROANA ROANO ROBAD ROBAN ROBAR ROBAS ROBEN ROBES ROBIN ROBLA ROBLE ROBLO ROBOS ROBOT ROBRA ROBRE ROCAS ROCEN ROCES ROCIA ROCIE ROCIN ROCIO RODAD RODAL RODAR RODAS RODEA RODEE RODEO RODIA RODIO ROEIS ROELA ROELE ROELO ROERA ROERE ROETE ROGAD ROGAR ROGUE ROIAN ROIAS ROIDA ROIDO ROIGA ROIGO ROJAL ROJAS ROJEA ROJEE ROJEO ROJEZ ROJOS ROLAD ROLAN ROLAR ROLAS ROLDA ROLDE ROLDO ROLEN ROLEO ROLES ROLLA ROLLE ROLLO ROMAS ROMBO ROMEA ROMEO ROMOS ROMPA ROMPE ROMPI ROMPO RONCA RONCE RONCO RONDA RONDE RONDO RONES RONZA RONZO ROPAS ROPON ROQUE RORAD RORAN RORAR RORAS ROREN RORES RORRO ROSAD ROSAL ROSAN ROSAR ROSAS ROSCA ROSCO ROSEA ROSEE ROSEN ROSEO ROSES ROSJO ROSON ROSOS ROTAD ROTAL ROTAN ROTAR ROTAS ROTEN ROTES ROTOR ROTOS ROYAN ROYAS ROYOS ROZAD ROZAN ROZAR ROZAS ROZNA ROZNE ROZNO ROZON ROZOS RUABA RUADA RUADO RUAIS RUANA RUANO RUARA RUARE RUASE RUBEA RUBEO RUBIA RUBIN RUBIO RUBLO RUBOR RUBOS RUBRA RUBRO RUCAD RUCAN RUCAR RUCAS RUCHA RUCHE RUCHO RUCIA RUCIO RUCOS RUDAS RUDOS RUECA RUEDA RUEDE RUEDO RUEGA RUEGO RUEIS RUEJO RUFAS RUFON RUFOS RUGAD RUGAN RUGAR RUGAS RUGBY RUGEN RUGES RUGIA RUGID RUGIO RUGIR RUGIS RUGUE RUIDO RUINA RUINE RUINO RUJAN RUJAS RUJIA RUJIE RUJIO RULAD RULAN RULAR RULAS RULEN RULES RULOS RUMBA RUMBE RUMBO RUMIA RUMIE RUMIO RUMOR RUMOS RUNAS RUNGO RUNOS RUPIA RUQUE RURAL RUSAS RUSCO RUSIA RUSOS RUSTA RUSTE RUSTI RUSTO RUTAD RUTAN RUTAR RUTAS RUTEL RUTEN RUTES SABED SABEN SABEO SABER SABES SABIA SABIO SABLE SABOR SABRA SABRE SACAD SACAN SACAR SACAS SACES SACHA SACHE SACHO SACIA SACIE SACIO SACOS SACRA SACRE SACRO SAETA SAETI SAGAS SAGAZ SAINA SAINE SAINO SAJAD SAJAN SAJAR SAJAS SAJEN SAJES SAJIA SAJON SALAD SALAN SALAR SALAS SALAZ SALCE SALDA SALDE SALDO SALEA SALEE SALEN SALEO SALEP SALES SALGA SALGO SALIA SALID SALIN SALIO SALIR SALIS SALLA SALLE SALLO SALMA SALME SALMO SALOL SALON SALPA SALSA SALTA SALTE SALTO SALUD SALVA SALVE SALVO SAMAN SAMAS SAMBA SAMIA SAMIO SAMPA SANAD SANAN SANAR SANAS SANCO SANEA SANEE SANEN SANEO SANES SANGO SANIE SANOS SANSA SANSO SANTA SANTO SAPOS SAQUE SARAN SARAO SARCO SARDA SARDE SARDO SARGA SARGO SARIA SARNA SARRO SARTA SATAN SATEN SATIN SATIS SATOS SAUCE SAUCO SAUDI SAUNA SAVIA SAXEA SAXEO SAXOS SAYAL SAYAS SAYON SAYOS SAZON SEAIS SEBES SEBOS SECAD SECAN SECAR SECAS SECON SECOS SECTA SECUA SEDAD SEDAL SEDAN SEDAR SEDAS SEDEA SEDEE SEDEN SEDEO SEDES SEGAD SEGAR SEGAS SEGRI SEGUE SEGUI SEGUN SEGUR SEICO SEISE SELLA SELLE SELLO SELVA SEMAS SEMEN SEMIS SENAS SENDA SENES SENIL SENOS SENTE SENTI SENTO SEPAN SEPAS SEPIA SEQUE SERAN SERAS SERES SERIA SERIE SERIO SERNA SERON SERPA SERRE SERRO SERVA SERVE SERVI SERVO SESEA SESEE SESEO SESGA SESGO SESMA SESMO SESOS SETAS SETOS SEUDO SEXMO SEXOS SEXTA SEXTO SIBIL SIDAS SIDOS SIDRA SIEGA SIEGO SIENA SIESO SIETE SIFON SIGAN SIGAS SIGLA SIGLO SIGMA SIGNA SIGNE SIGNO SIGUE SILBA SILBE SILBO SILEX SILFO SILLA SILOS SILVA SIMAS SIMIA SIMIL SIMIO SIMON SINOS SIRAS SIRGA SIRGO SIRIA SIRIO SIRLE SIROS SIRTE SIRVA SIRVE SIRVO SISAD SISAN SISAR SISAS SISEA SISEE SISEN SISEO SISES SISMO SISON SITAS SITIA SITIE SITIO SITOS SITUA SITUE SITUO SOASA SOASE SOASO SOBAD SOBAN SOBAR SOBAS SOBEN SOBEO SOBES SOBON SOBOS SOBRA SOBRE SOBRO SOCAS SOCAZ SOCIA SOCIO SODAS SODIO SOEZA SOFAS SOGAS SOJAS SOLAD SOLAR SOLAS SOLAZ SOLDE SOLDO SOLEA SOLED SOLEE SOLEO SOLER SOLES SOLFA SOLIA SOLIO SOLLA SOLOS SOLTE SOLTO SOMAS SOMOS SONAD SONAR SONDA SONDE SONDO SONES SONIA SONSO SOPAD SOPAN SOPAR SOPAS SOPEA SOPEE SOPEN SOPEO SOPES SOPLA SOPLE SOPLO SOPON SOPOR SORBA SORBE SORBI SORBO SORCE SORDA SORDO SORGO SORNA SORNE SORNO SOROS SORRA SOSAR SOSAS SOSIA SOSOS SOTAS SOTES SOTOL SOTOS SOVOZ SPORT STAND SUAVE SUBAN SUBAS SUBEN SUBES SUBIA SUBID SUBIO SUBIR SUBIS SUCIA SUCIO SUCRE SUDAD SUDAN SUDAR SUDAS SUDEN SUDES SUDOR SUECA SUECO SUELA SUELE SUELO SUENA SUENE SUENO SUERO SUEVA SUEVO SUFRA SUFRE SUFRI SUFRO SUITE SUIZA SUIZO SULAS SUMAD SUMAN SUMAR SUMAS SUMEN SUMES SUMIA SUMID SUMIO SUMIR SUMIS SUMOS SUPER SUPLA SUPLE SUPLI SUPLO SUPON SUPRA SURAS SURCA SURCO SURGE SURGI SURJA SURJO SURTA SURTE SURTI SURTO SUSTO SUTAS SUTES SUTIL SUYAS SUYOS SUZON TABAS TABEA TABLA TACAD TACAN TACAR TACAS TACEN TACES TACHA TACHE TACHO TACON TACOS TACTO TAFIA TAFON TAFOS TAGUA TAHAS TAHUR TAIFA TAIGA TAIMA TAINA TAINO TAJAD TAJAN TAJAR TAJAS TAJEA TAJEN TAJES TAJON TAJOS TALAD TALAN TALAR TALAS TALCO TALEA TALEN TALES TALIN TALIO TALLA TALLE TALLO TALMA TALON TALOS TALPA TALUD TAMAL TAMBA TAMBO TAMIZ TAMOS TANDA TANGA TANGO TANKA TANOR TANTA TANTO TAPAD TAPAN TAPAR TAPAS TAPEA TAPEE TAPEN TAPEO TAPES TAPIA TAPIE TAPIN TAPIO TAPIR TAPIS TAPIZ TAPON TAQUE TARAD TARAN TARAR TARAS TARAY TARCO TARDA TARDE TARDO TAREA TAREN TARES TARIN TARJA TARJE TARJO TAROT TARRO TARSO TARTA TASAD TASAN TASAR TASAS TASCA TASCO TASEN TASES TASIA TASIO TASTO TATAS TATES TATOS TATUA TATUE TATUO TAULA TAURO TAXIS TAXON TAZAD TAZAN TAZAR TAZAS TAZON TBYTE TEAME TEBEO TECAS TECES TECHA TECHE TECHO TECLA TEDAS TEDIO TEFES TEGEA TEGEO TEGUA TEJAD TEJAN TEJAR TEJAS TEJED TEJEN TEJER TEJES TEJIA TEJIO TEJON TEJOS TELAR TELAS TELON TEMAN TEMAS TEMED TEMEN TEMER TEMES TEMIA TEMIO TEMOR TEMPO TENAS TENAZ TENDI TENED TENER TENGA TENGO TENIA TENIS TENLA TENLE TENLO TENME TENOR TENSA TENSE TENSO TENTE TENTO TENUE TEOSA TEOSO TERCA TERCO TERMA TERMO TERNA TERNE TERNO TEROS TERSA TERSE TERSO TESAD TESAN TESAR TESAS TESEN TESES TESIS TESLA TESON TESOS TESTA TESTE TESTO TETAD TETAN TETAR TETAS TETEN TETES TETON TETRA TEXAS TEXTO TEYAS TEYOS TIACA TIARA TIBES TIBIA TIBIE TIBIO TIENE TIESA TIESO TIFON TIFOS TIFUS TIGRE TIJAS TILAS TILDA TILDE TILDO TILIA TILIN TILLA TILLE TILLO TILMA TILOS TIMAD TIMAN TIMAR TIMAS TIMBA TIMBO TIMEN TIMES TIMOL TIMON TIMOS TIMPA TINAS TINCA TINGE TINOS TINTA TINTE TINTO TIPAS TIPIS TIPLE TIPOS TIQUE TIRAD TIRAN TIRAR TIRAS TIREN TIRES TIRIA TIRIO TIRON TIROS TIRSO TISIS TISTE TITAD TITAN TITAR TITAS TITEN TITES TITOS TIZAS TIZNA TIZNE TIZNO TIZON TIZOS TLACO TOABA TOADA TOADO TOAIS TOARA TOARE TOASE TOBAR TOBAS TOCAD TOCAN TOCAR TOCAS TOCEN TOCES TOCHA TOCHE TOCHO TOCIA TOCIO TOCON TOCOS TODAS TODOS TOEIS TOESA TOFOS TOGAS TOJAL TOJOS TOKEN TOLAS TOLDA TOLDE TOLDO TOLES TOLLA TOLLO TOLMO TOLON TOLVA TOMAD TOMAN TOMAR TOMAS TOMEN TOMES TOMON TOMOS TONAD TONAL TONAN TONAR TONAS TONCA TONDO TONEL TONEN TONER TONES TONGA TONGO TONOS TONTA TONTO TOPAD TOPAN TOPAR TOPAS TOPEN TOPES TOPOS TOQUE TORAL TORAS TORAX TORCA TORCE TORCI TORCO TORDA TORDO TOREA TOREE TOREO TORGA TORGO TORIL TORIO TORMO TORNA TORNE TORNO TOROS TORPE TORRA TORRE TORRO TORSO TORTA TORVA TORVO TOSAN TOSAS TOSCA TOSCO TOSED TOSEN TOSER TOSES TOSIA TOSIO TOSTE TOSTO TOTAL TOTEM TOVAS TOZAD TOZAN TOZAR TOZAS TOZOS TRABA TRABE TRABO TRACA TRACE TRAED TRAEN TRAER TRAES TRAGA TRAGO TRAIA TRAJE TRAJO TRAMA TRAME TRAMO TRAPA TRAPE TRAPO TRAPS TRARO TRATA TRATE TRATO TRAZA TRAZO TREBO TRECE TREJA TREMA TREME TREMI TREMO TRENA TRENO TREOS TREPA TREPE TREPO TRETA TRIAD TRIAL TRIAN TRIAR TRIAS TRIBU TRICE TRIEN TRIES TRIGA TRIGO TRILE TRINA TRINE TRINO TRIOS TRIPA TRIPE TRISA TRIZA TRIZO TROCE TROCO TROJE TROLA TROLE TRONA TRONE TRONO TROPA TROPO TROTA TROTE TROTO TROVA TROVE TROVO TROYA TROZA TROZO TRUCA TRUCO TRUFA TRUFE TRUFO TRUJA TRUSA TRUST TUBAS TUBOS TUCAN TUCAS TUCOS TUDAS TUECA TUECO TUERA TUFOS TULES TULIO TULLA TULLE TULLI TULLO TUMBA TUMBE TUMBO TUMOR TUMOS TUNAD TUNAN TUNAR TUNAS TUNCO TUNDA TUNDE TUNDI TUNDO TUNEA TUNEE TUNEL TUNEN TUNEO TUNES TUNEZ TUNOS TUPAN TUPAS TUPEN TUPES TUPIA TUPID TUPIN TUPIO TUPIR TUPIS TUPLA TURBA TURBE TURBO TURCA TURCO TURMA TURNA TURNE TURNO TURON TURRA TURRE TURRO TUSAS TUSON TUTEA TUTEE TUTEO TUTES TUTOR TUYAS TUYOS UBICA UBICO UBIOS UBRES UCASE UFANA UFANE UFANO UGRES UJIER ULEMA ULTRA ULULA ULULE ULULO UMERO UNAIS UNALA UNALE UNALO UNAME UNASE UNCEN UNCES UNCIA UNCID UNCIO UNCIR UNCIS UNELA UNELE UNELO UNEME UNETE UNGEN UNGES UNGIA UNGID UNGIO UNGIR UNGIS UNIAN UNIAS UNICA UNICO UNIDA UNIDO UNION UNIOS UNIRA UNIRE UNJAN UNJAS UNTAD UNTAN UNTAR UNTAS UNTEN UNTES UNTOS UNZAN UNZAS URANO URAPE URATO URBES URDAN URDAS URDEN URDES URDIA URDID URDIO URDIR URDIS UREAS URGEN URGES URGIA URGID URGIO URGIR URGIS URICA URICO URJAN URJAS URNAS URUBU URUCU URUGA USABA USADA USADO USAIS USAJE USALA USALE USALO USAME USAOS USARA USARE USART USASE USATE USEIS USELA USELE USELO USEME USESE USIAS USTED USUAL USURA USURE USURO UTERO UVADA UVEAS UVERA UVERO UVULA VACAD VACAN VACAR VACAS VACIA VACIE VACIO VACOS VACUA VACUO VADEA VADEE VADEO VADES VADOS VAGAD VAGAN VAGAR VAGAS VAGON VAGOS VAGUE VAHOS VAIDA VAINA VALAR VALED VALEN VALER VALES VALGA VALGO VALIA VALIO VALLA VALLE VALLO VALON VALOR VALSA VALSE VALSO VALUA VALUE VALUO VALVA VAMOS VANAS VANEA VANEE VANEO VANOS VAPOR VAQUE VARAD VARAL VARAN VARAR VARAS VAREA VAREE VAREN VAREO VARES VARGA VARIA VARIE VARIO VARIZ VARON VASAR VASCA VASCO VASOS VASTA VASTO VATER VATES VATIO VAYAN VAYAS VEAIS VEALA VEALE VEALO VEAME VEASE VECEN VECES VEDAD VEDAN VEDAR VEDAS VEDEN VEDES VEDLA VEDLE VEDLO VEDME VEGAS VEIAN VEIAS VEJAD VEJAN VEJAR VEJAS VEJEN VEJES VEJEZ VELAD VELAN VELAR VELAS VELEN VELES VELLO VELON VELOS VELOZ VEMOS VENAL VENAS VENCE VENCI VENDA VENDE VENDI VENDO VENGA VENGO VENIA VENID VENIR VENIS VENME VENOS VENTA VENTE VENTO VENUS VENZA VENZO VERAN VERAS VERAZ VERBO VERDE VERGA VERGE VERIA VERIL VERJA VERLA VERLE VERLO VERME VERMU VEROS VERSA VERSE VERSO VERTE VERTI VESTE VESTI VETAD VETAN VETAR VETAS VETEA VETEE VETEN VETEO VETES VETOS VEZAD VEZAN VEZAR VEZAS VIADA VIAJA VIAJE VIAJO VIBRA VIBRE VIBRO VICES VICIA VICIE VICIO VICTO VIDAS VIDEO VIDES VIEJA VIEJO VIENE VIERA VIERE VIESE VIGAS VIGIA VIGIE VIGIO VIGOR VILES VILLA VILOS VIMOS VINAR VINCO VINOS VINTA VIOLA VIOLE VIOLO VIRAD VIRAL VIRAN VIRAR VIRAS VIREN VIRES VIRGO VIRIL VIRON VIRUS VISAD VISAN VISAR VISAS VISCO VISEA VISEE VISEN VISEO VISES VISIR VISON VISOR VISOS VISTA VISTE VISTO VITAE VITAL VITOR VITOS VITRE VIUDA VIUDO VIVAC VIVAN VIVAR VIVAS VIVAZ VIVEN VIVES VIVIA VIVID VIVIO VIVIR VIVIS VIVOS VOCAL VOCEA VOCEE VOCEO VOCES VODKA VOLAD VOLAR VOLCO VOLEA VOLEE VOLEO VOLVI VORAZ VOSEA VOSEE VOSEO VOTAD VOTAN VOTAR VOTAS VOTEN VOTES VOTOS VUELA VUELE VUELO VULGO VULVA WEBER YABAS YACED YACEN YACER YACES YACIA YACIO YAGAN YAGAS YAMBO YANAS YAQUI YARDA YATES YAYAS YAYOS YAZCA YAZCO YAZGA YAZGO YEBOS YECOS YEDRA YEGUA YELMO YEMAS YENDO YENES YERBA YERMA YERME YERMO YERNO YEROS YERRA YERRE YERRO YERTA YERTO YERVO YESAR YESCA YESON YESOS YETIS YEZGO YODOS YOGAS YOGUR YOYOS YUGOS YUNTA YUNTO YUTES ZAFAD ZAFAN ZAFAR ZAFAS ZAFEN ZAFES ZAFIA ZAFIO ZAFOS ZAFRA ZAGAL ZAGAS ZAHON ZAINA ZAINO ZAMBO ZAMPA ZAMPE ZAMPO ZANCA ZANCO ZANGA ZANJA ZANJE ZANJO ZAPAD ZAPAN ZAPAR ZAPAS ZAPEA ZAPEE ZAPEN ZAPEO ZAPES ZAQUE ZARBO ZARCA ZARCO ZARES ZARJA ZARPA ZARPE ZARPO ZARZA ZARZO ZATAS ZAYAS ZEDAS ZENDA ZENDO ZENIT ZETAS ZILOG ZOCAS ZOCOS ZOFRA ZOIZO ZOMAS ZOMPA ZOMPO ZONAL ZONAS ZOPOS ZORRA ZORRO ZOTES ZUAVO ZUBIA ZUDAS ZUECO ZUELA ZUIZA ZULOS ZUMBA ZUMBE ZUMBO ZUMOS ZURCE ZURCI ZURDA ZURDO ZUREA ZUREE ZUREO ZUROS ZURRA ZURRE ZURRI ZURRO ZURZA ZURZO".split(
    " ")

#np.random.seed(42)
s = sorted(np.random.choice(s, l)) 
print(len(s))

In [ ]:
import re
import math

def generatePattern(entrada, word):
  pattern = ""
  procesed = {}

  for j in range(len(entrada)):
    letra = word[j]
    if letra not in procesed:
      condition = max([word[k]==letra and entrada[k]==2 for k in range(j+1, len(entrada))]+[False])
      if entrada[j]==0 and not condition:
        procesed[letra] = 0
      pattern += [f"(?=[^{letra}]*$)" if not condition else "", f"(?!.{{{j}}}{letra})(?=.*{letra})", f"(?=.{{{j}}}{letra})"][entrada[j]]
  return f"^{pattern}.*$"

def scoreWord(word, d):191869
  combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101", "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210", "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012", "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121", "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000", "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102", "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211", "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020", "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122", "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001", "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110", "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212", "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021", "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200", "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002", "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111", "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220", "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022", "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201", "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010", "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112", "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221", "22222"]
  finalScore = 0

  for c in combinations:
    entrada = [int(i) for i in c]
    pattern = generatePattern(entrada, word)
    
    p = 0
    for i in d.keys(): p+=1 if re.match(pattern, i) else 0
    p/=len(d)
    finalScore += p*math.log(p,2) if p>0 else 0
  return finalScore

def updateDict(d, pattern):
  d={k:0 for (k, v) in d.items() if re.match(pattern, k)}
  return {k:scoreWord(k, d) for (k, v) in d.items()}

intentos = 6
d = dict.fromkeys(sorted(s), 0)
d = updateDict(d, f"^[A-Z][A-Z][A-Z][A-Z][A-Z]$")
for i in range(intentos):
  print(len(d), d)
  word = max(d, key=d.get)
  print(word)

  entrada = []
  while len(entrada)!=5:
    entrada = [int(a) for a in input()[:5] if int(a) in range(0,3)]

  pattern = generatePattern(entrada, word)
  d = updateDict(d, pattern)

**Versión multithreading**

In [ ]:
import re
import math
import os
import threading

class ThreadWithReturnValue(threading.Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        threading.Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        threading.Thread.join(self, *args)
        return self._return

def procesarEntrada():
  entrada = []
  while len(entrada)!=5:
    entrada = [int(a) for a in input()[:5] if int(a) in range(0,3)]
  return entrada

def scoreWord(word, d):
  combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101", "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210", "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012", "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121", "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000", "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102", "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211", "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020", "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122", "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001", "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110", "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212", "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021", "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200", "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002", "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111", "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220", "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022", "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201", "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010", "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112", "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221", "22222"]
  finalScore = 0

  for c in combinations:
    entrada = [int(i) for i in c]
    pattern = ""
    procesed = {}

    for j in range(len(entrada)):
      letra = word[j]
      if letra not in procesed:
        condition = max([word[k]==letra and entrada[k]==2 for k in range(j+1, len(entrada))]+[False])
        if entrada[j]==0 and not condition:
          procesed[letra] = 0
        pattern += [f"(?=[^{letra}]*$)" if not condition else "", f"(?!.{{{j}}}{letra})(?=.*{letra})", f"(?=.{{{j}}}{letra})"][entrada[j]]
    p = 0
    for i in d.keys(): p+=1 if re.match(pattern, i) else 0
    p/=len(d)
    finalScore += p*math.log(p,2) if p>0 else 0
  return finalScore

def paralelDict(chunk, completeDict):
  return {i:scoreWord(i,completeDict) for i in chunk}

def updateDict(d, pattern):
  d={k:0 for (k, v) in d.items() if re.match(pattern, k)}

  n = os.cpu_count()
  chunkSize = math.ceil(len(d)/n)
  l = []
  for i in range(n):
    t = ThreadWithReturnValue(target=paralelDict, args=(list(d.keys())[chunkSize*i:chunkSize*(i+1)], d,))
    l.append(t)
    t.start()
  out = {}
  for i in l:
    dictionary = i.join()
    out.update(dictionary)
  return out

intentos = 6
d = dict.fromkeys(sorted(s), 0)
d = updateDict(d, f"^[A-Z][A-Z][A-Z][A-Z][A-Z]$")
for i in range(intentos):
  print(len(d), d)
  word = max(d, key=d.get)
  print(word)

  entrada = procesarEntrada()
  pattern = ""
  procesed = {}

  for j in range(len(entrada)):
    letra = word[j]
    if letra not in procesed:
      condition = max([word[k]==letra and entrada[k]==2 for k in range(j+1, len(entrada))]+[False])
      if entrada[j]==0 and not condition:
        procesed[letra] = 0
      pattern += [f"(?=[^{letra}]*$)" if not condition else "", f"(?!.{{{j}}}{letra})(?=.*{letra})", f"(?=.{{{j}}}{letra})"][entrada[j]]
  d = updateDict(d, f"^{pattern}.*$")

**Versión buena (FINAL FINAL) multiprocessing**

In [ ]:
import re
import math
import os
import concurrent
from concurrent.futures import ProcessPoolExecutor
import requests
import numpy as np


def procesarEntrada():
    entrada = []
    while len(entrada) != 5:
        entrada = [int(a) for a in input()[:5] if int(a) in range(0, 3)]
    return entrada


def generatePattern(entrada, word):
    pattern = ""
    procesed = {}

    for j in range(len(entrada)):
        letra = word[j]
        if letra not in procesed:
            condition = [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] == 2]
            if entrada[j] == 0:
                if condition == []: procesed[letra] = 0
            else:
                procesed[letra] = 0
            pattern += [f"(?=[^{letra}]*$)" if condition == [] else f"(?!.{{{j}}}{letra})",
                        f"(?!.{{{j}}}{letra})(?=.*{letra})" + "".join(f"(?!.{{{i}}}{letra})" for i in [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] in [0, 1]]),
                        f"(?=.{{{j}}}{letra})" + "".join(f"(?!.{{{i}}}{letra})" for i in [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] in [0, 1]])][
                entrada[j]]
    return f"^{pattern}.*$"


def scoreWord(word, d):
    global combinations
    finalScore = 0

    for c in combinations:
        entrada = [int(i) for i in c]
        pattern = generatePattern(entrada, word)
        p = 0
        for i in d.keys(): p += 1 if re.match(pattern, i) else 0
        p /= len(d)
        finalScore += p * math.log(p, 2) if p > 0 else 0
    # print(f"{word}:{finalScore}")
    return finalScore


def paralelDict(item, d):
    return {i: scoreWord(i, d) for i in item}


def updateDict(d, pattern):
    d = {k: 0 for (k, v) in d.items() if re.match(pattern, k)}

    n = os.cpu_count()
    chunkSize = math.ceil(len(d) / n)
    out = {}
    with ProcessPoolExecutor(n) as executor:
        futures = [executor.submit(paralelDict, list(d.keys())[chunkSize * i:chunkSize * (i + 1)], d) for i in range(n)]
        for future in concurrent.futures.as_completed(futures):
            out.update(future.result())
        executor.shutdown()
    return out


def validarEntrada(entrada, word, globalPattern):
    procesed = {}
    for i in range(len(entrada)):
        letra = word[i]
        if letra not in procesed:
            if entrada[i] == 0:
                if (f"(?=.{{{i}}}{letra})" in globalPattern) or (f"(?=.*{letra})" in globalPattern and not max(
                        [entrada[j] == 2 and word[j] == letra for j in range(i + 1, len(entrada))] + [False])) or max([entrada[j] == 1 and word[j] == letra for j in range(i + 1, len(entrada))] + [False]):
                    print(f"Error en 0 letra {letra}")
                    return False
            elif entrada[i] == 1:
                huecos = sum([1 for k in range(0, len(entrada)) if not(word[k] == letra and entrada[k] == 1) and not entrada[k] == 2])
                if f"(?=.{{{i}}}{letra})" in globalPattern or sum([1 for k in range(i, len(entrada)) if word[k] == letra and entrada[k] == 1]) > huecos:
                    return False
            elif entrada[i] == 2:
                if f"(?!.{{{i}}}{letra})" in globalPattern or f"(?=[^{letra}]*$)" in globalPattern:
                    print(f"Error en 2 letra {letra}")
                    return False
            procesed[letra] = 0
    return True


if __name__ == '__main__':
    combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101",
                    "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210",
                    "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012",
                    "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121",
                    "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000",
                    "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102",
                    "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211",
                    "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020",
                    "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122",
                    "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001",
                    "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110",
                    "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212",
                    "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021",
                    "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200",
                    "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002",
                    "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111",
                    "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220",
                    "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022",
                    "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201",
                    "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010",
                    "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112",
                    "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221",
                    "22222"]
    l = 200
    s = requests.get(
        "https://media.githubusercontent.com/media/cardstdani/practica-java/main/Data/Diccionario.txt").text.split(" ")
    #np.random.seed(52)
    s = sorted(np.random.choice(s, l))

    intentos = 6
    d = dict.fromkeys(sorted(s), 0)
    d = updateDict(d, f"^[A-Z][A-Z][A-Z][A-Z][A-Z]$")
    globalPattern = ""
    for intento in range(intentos):
        print(len(d), d, len(d))
        word = "ABABA" if intento==0 else max(d, key=d.get)
        print(word)

        entrada = procesarEntrada()
        pattern = generatePattern(entrada, word)
        if validarEntrada(entrada, word, globalPattern):
            globalPattern += pattern[1:-3]
            d = updateDict(d, pattern)
        else:
            print("Error detectado, entrada inconsistente")
            intento -= 1
        if entrada == [2, 2, 2, 2, 2]:
            break

In [ ]:
generatePattern([0,1,2,1,0], "ARCAS")

**Multiprocessing version, with precalculated data (FINAL FINAL FINAL)**

In [ ]:
import re
import math
import json
import os
import concurrent
from concurrent.futures import ProcessPoolExecutor
import requests
import numpy as np


def procesarEntrada():
    entrada = []
    while len(entrada) != 5:
        entrada = [int(a) for a in input()[:5] if int(a) in range(0, 3)]
    return entrada


def generatePattern(entrada, word):
    pattern = ""
    procesed = {}

    for j in range(len(entrada)):
        letra = word[j]
        if letra not in procesed:
            condition = [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] == 2]
            if entrada[j] == 0:
                if condition == []: procesed[letra] = 0
            else:
                procesed[letra] = 0
            pattern += [f"(?=[^{letra}]*$)" if condition == [] else f"(?!.{{{j}}}{letra})",
                        f"(?!.{{{j}}}{letra})(?=.*{letra})" + "".join(f"(?!.{{{i}}}{letra})" for i in [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] in [0, 1]]),
                        f"(?=.{{{j}}}{letra})" + "".join(f"(?!.{{{i}}}{letra})" for i in [k for k in range(j + 1, len(entrada)) if word[k] == letra and entrada[k] in [0, 1]])][
                entrada[j]]
    return f"^{pattern}.*$"


def scoreWord(word, d):
    combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101",
                    "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210",
                    "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012",
                    "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121",
                    "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000",
                    "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102",
                    "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211",
                    "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020",
                    "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122",
                    "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001",
                    "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110",
                    "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212",
                    "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021",
                    "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200",
                    "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002",
                    "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111",
                    "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220",
                    "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022",
                    "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201",
                    "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010",
                    "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112",
                    "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221",
                    "22222"]
    finalScore = 0

    for c in combinations:
        entrada = [int(i) for i in c]
        pattern = generatePattern(entrada, word)
        p = 0
        for i in d.keys(): p += 1 if re.match(pattern, i) else 0
        p /= len(d)
        finalScore += p * math.log(p, 2) if p > 0 else 0
    # print(f"{word}:{finalScore}")
    return finalScore


def paralelDict(item, d):
    return {i: scoreWord(i, d) for i in item}


def updateDict(d, pattern):
    d = {k: 0 for (k, v) in d.items() if re.match(pattern, k)}

    n = os.cpu_count()
    chunkSize = math.ceil(len(d) / n)
    out = {}
    with ProcessPoolExecutor(n) as executor:
        futures = [executor.submit(paralelDict, list(d.keys())[chunkSize * i:chunkSize * (i + 1)], d) for i in range(n)]
        for future in concurrent.futures.as_completed(futures):
            out.update(future.result())
        executor.shutdown()
    return out


def validarEntrada(entrada, word, globalPattern):
    procesed = {}
    for i in range(len(entrada)):
        letra = word[i]
        if letra not in procesed:
            if entrada[i] == 0:
                if (f"(?=.{{{i}}}{letra})" in globalPattern) or (f"(?=.*{letra})" in globalPattern and not max(
                        [entrada[j] == 2 and word[j] == letra for j in range(i + 1, len(entrada))] + [False])) or max([entrada[j] == 1 and word[j] == letra for j in range(i + 1, len(entrada))] + [False]):
                    print(f"Error en 0 letra {letra}")
                    return False
            elif entrada[i] == 1:
                if f"(?=.{{{i}}}{letra})" in globalPattern:
                    return False
            elif entrada[i] == 2:
                if f"(?!.{{{i}}}{letra})" in globalPattern or f"(?=[^{letra}]*$)" in globalPattern:
                    print(f"Error en 2 letra {letra}")
                    return False
            procesed[letra] = 0
    return True


if __name__ == '__main__':
    intentos = 6
    d = json.loads(requests.get("https://media.githubusercontent.com/media/cardstdani/practica-java/main/Data/DictScoreData.txt").text)
    globalPattern = ""
    for intento in range(intentos):
        print(len(d), d, len(d))
        word = max(d, key=d.get)
        print(word, d[word])

        entrada = procesarEntrada()
        pattern = generatePattern(entrada, word)
        if validarEntrada(entrada, word, globalPattern):
            globalPattern += pattern[1:-3]
            try:
              d=json.loads(requests.get(f"https://media.githubusercontent.com/media/cardstdani/practica-java/main/Data/MaxTree/Dict{intento+1}-{''.join([str(a) for a in entrada])}.txt").text)
            except:
              d = updateDict(d, pattern)
        else:
            print("Error detectado, entrada inconsistente")
            intento -= 1
        if entrada == [2, 2, 2, 2, 2]:
            break

##**Data Analysis**

In [ ]:
import json
import requests
import matplotlib.pyplot as plt

response = requests.get("https://media.githubusercontent.com/media/cardstdani/practica-java/main/Data/DictScoreData.txt")
d = json.loads(response.text)

print(len(d), sorted(d.items(), key=lambda x:x[1]))
fig = plt.figure(figsize=(10, 10))
plt.plot(list(range(len(d))), [-i for i in sorted(list(d.values()))])

In [ ]:
import json
import requests
import matplotlib.pyplot as plt

response = requests.get("https://raw.githubusercontent.com/cardstdani/practica-java/main/Data/patterns.txt")
d = json.loads(response.text)

print(len(d), sorted(d.items(), key=lambda x:x[1]))
fig = plt.figure(figsize=(10, 10))
plt.plot(list(range(len(d))), [-i for i in sorted(list(d.values()))])

**Generate all patterns**

In [ ]:
import json

s = "ABABA ABACO ABADA ABADO ABAIS ABAJA ABAJE ABAJO ABANA ABANE ABANO ABAOS ABARA ABARE ABASE ABATA ABATE ABATI ABATO ABECE ABEIS ABEJA ABETO ABIAR ABOBA ABOBE ABOBO ABOCA ABOCO ABOGA ABOGO ABOLA ABOLE ABOLI ABOLO ABONA ABONE ABONO ABOYA ABOYE ABOYO ABRAN ABRAS ABREN ABRES ABRIA ABRID ABRIL ABRIO ABRIR ABRIS ABUCE ABUJE ABUSA ABUSE ABUSO ABUZA ABUZO ACABA ACABE ACABO ACAMA ACAME ACAMO ACANA ACARO ACASO ACATA ACATE ACATO ACEBO ACECE ACEDA ACEDE ACEDO ACEPA ACEPE ACEPO ACERA ACERE ACERO ACEZA ACEZO ACIDA ACIDO ACIMO ACLES ACODA ACODE ACODO ACOGE ACOGI ACOJA ACOJO ACORO ACOSA ACOSE ACOSO ACOTA ACOTE ACOTO ACREA ACREE ACREI ACREO ACRES ACTAS ACTEA ACTOR ACTOS ACTUA ACTUE ACTUO ACUDA ACUDE ACUDI ACUDO ACUEA ACUEO ACULA ACULE ACULO ACUNA ACUNE ACUNO ACURE ACUSA ACUSE ACUSO ACUTI ADAMA ADAME ADAMO ADIOS ADOBA ADOBE ADOBO ADORA ADORE ADORO ADOSA ADOSE ADOSO ADRAD ADRAN ADRAR ADRAS ADREN ADRES ADUAR ADUCE ADUJE ADUJO ADULA ADULE ADULO ADVEN AEDOS AEREA AEREO AETAS AFAMA AFAME AFAMO AFANA AFANE AFANO AFATA AFATE AFATO AFEAD AFEAN AFEAR AFEAS AFEEN AFEES AFIJA AFIJO AFILA AFILE AFILO AFINA AFINE AFINO AFLUI AFORA AFORE AFORO AFROS AFTAS AGAPE AGATA AGITA AGITE AGITO AGORA AGORE AGORO AGOTA AGOTE AGOTO AGRAZ AGRIA AGRIE AGRIO AGUAD AGUAN AGUAR AGUAS AGUCE AGUDA AGUDO AGUEN AGUES AGUJA AGUZA AGUZO AHAJA AHAJE AHAJO AHIJA AHIJE AHIJO AHILA AHILE AHILO AHITA AHITE AHITO AHOGA AHOGO AHORA AHOYA AHOYE AHOYO AHUMA AHUME AHUMO AHUSA AHUSE AHUSO AINAS AIRAD AIRAR AIREA AIREE AIREO AIRES AIRON AISLA AISLE AISLO AJABA AJADA AJADO AJAIS AJALA AJALE AJALO AJAME AJAOS AJARA AJARE AJASE AJATE AJEAD AJEAN AJEAR AJEAS AJEEN AJEES AJEIS AJELA AJELE AJELO AJEME AJENA AJENO AJEOS AJERA AJERO AJESE AJETE AJORA AJORE AJORO AJOTE AJUAR AJUMA AJUME AJUMO ALABA ALABE ALABO ALADA ALADO ALAFA ALAGA ALAGO ALAJU ALAMA ALAMO ALANO ALAUI ALAZO ALBAR ALBAS ALBIN ALBOR ALBOS ALBUM ALBUR ALCEA ALCEN ALCES ALCOR ALDEA ALEAD ALEAN ALEAR ALEAS ALECE ALEDA ALEEN ALEES ALEGA ALEGO ALEJA ALEJE ALEJO ALELA ALELE ALELI ALELO ALERO ALETA ALETO ALEYA ALEZO ALFAD ALFAN ALFAR ALFAS ALFEN ALFES ALFIL ALFIZ ALFOZ ALGAR ALGAS ALGOL ALGUN ALIAD ALIAN ALIAR ALIAS ALICA ALIEN ALIES ALIFA ALIGA ALIGO ALIJA ALIJE ALIJO ALIMO ALISA ALISE ALISO ALJEZ ALLAS ALMAS ALMOS ALMUD ALOCA ALOCO ALOJA ALOJE ALOJO ALOMA ALOME ALOMO ALOSA ALOTA ALOTE ALOTO ALOYA ALPES ALTAR ALTAS ALTEA ALTEE ALTEO ALTOS ALUDA ALUDE ALUDI ALUDO ALVEA ALVEO ALZAD ALZAN ALZAR ALZAS AMABA AMADA AMADO AMAGA AMAGO AMAIS AMALA AMALE AMALO AMAME AMAOS AMARA AMARE AMARO AMASA AMASE AMASO AMATE AMBAR AMBAS AMBLA AMBLE AMBLO AMBON AMBOS AMEBA AMEIS AMELA AMELE AMELO AMEME AMENA AMENO AMEOS AMERA AMERE AMERO AMESE AMIAS AMIBA AMIBO AMIDA AMIGA AMIGO AMINO AMIRI AMITO AMOLA AMOLE AMOLO AMOMO AMPON AMPRA AMPRE AMPRO AMULA AMULE AMULO AMURA AMURE AMURO AMUSO ANACO ANADE ANAFE ANATA ANCAS ANCHA ANCHE ANCHO ANCLA ANCLE ANCLO ANCON ANDAD ANDAN ANDAR ANDAS ANDEN ANDES ANDON ANEAS ANEGA ANEGO ANEJA ANEJE ANEJO ANETO ANEXA ANEXE ANEXO ANGEL ANGLA ANGLO ANIDA ANIDE ANIDO ANIMA ANIME ANIMO ANION ANITO ANODO ANONA ANOTA ANOTE ANOTO ANSAR ANSAS ANSIA ANSIE ANSIO ANTES ANTIA ANTRO ANUAL ANUDA ANUDE ANUDO ANULA ANULE ANULO ANUOS ANURA ANURO AOJAD AOJAN AOJAR AOJAS AOJEN AOJES AORTA AOVAD AOVAN AOVAR AOVAS AOVEN AOVES APAGA APAGO APARA APARE APARO APEAD APEAN APEAR APEAS APEEN APEES APEGA APEGO APELA APELE APELO APENA APENE APENO APEOS APERA APERE APERO APICE APILA APILE APILO APIOS APNEA APOCA APOCO APODA APODE APODO APOYA APOYE APOYO APROA APROE APROO APTAS APTOS APURA APURE APURO AQUEL AQUEO ARABA ARABE ARABO ARADA ARADO ARAIS ARALA ARALE ARALO ARAME ARANA ARAOS ARARA ARARE ARASE ARATE ARAZA ARBOL ARCAD ARCAN ARCAR ARCAS ARCEA ARCEN ARCES ARCHA ARCON ARCOS ARDAN ARDAS ARDED ARDEN ARDER ARDES ARDIA ARDID ARDIO ARDOR ARDUA ARDUO AREAS AREIS ARELA ARELE ARELO AREME ARENA ARENE ARENO ARESE ARETE ARGEL ARGON ARGOS ARGOT ARGUI ARIAS ARICA ARICO ARIDA ARIDO ARIES ARIOS ARMAD ARMAN ARMAR ARMAS ARMEN ARMES ARNES AROMA AROME AROMO ARPAD ARPAN ARPAR ARPAS ARPEN ARPEO ARPES ARPIA ARPON ARQUE ARRAS ARREA ARREE ARREO ARRES ARRIA ARRIE ARRIO ARROZ ARRUA ARRUE ARRUO ARTES ARZON ASABA ASADA ASADO ASAIS ASALA ASALE ASALO ASAME ASAOS ASARA ASARE ASASE ASATE ASCII ASCOS ASCUA ASEAD ASEAN ASEAR ASEAS ASEDA ASEDE ASEDO ASEEN ASEES ASEIS ASELA ASELE ASELO ASEME ASEOS ASESA ASESE ASESO ASIAN ASIAS ASIDA ASIDO ASILA ASILE ASILO ASINA ASIOS ASIRA ASIRE ASMAS ASNAL ASNOS ASOLA ASOLE ASOLO ASOMA ASOME ASOMO ASONE ASONO ASPAS ASPID ASTAN ASTAS ASTED ASTEN ASTER ASTES ASTIA ASTIO ASTRO ASTUR ASUMA ASUME ASUMI ASUMO ASURA ASURE ASURO ATABA ATABE ATACA ATACO ATADA ATADO ATAIS ATAJA ATAJE ATAJO ATALA ATALE ATALO ATAME ATAOS ATARA ATARE ATASE ATATE ATAUD ATEAS ATECE ATEIS ATELA ATELE ATELO ATEME ATEOS ATESE ATETA ATETE ATETO ATEZA ATEZO ATIBA ATIBE ATIBO ATICA ATICE ATICO ATINA ATINE ATINO ATIZA ATIZO ATLAS ATOAD ATOAN ATOAR ATOAS ATOEN ATOES ATOMO ATONA ATONO ATORA ATORE ATORO ATRAE ATRAS ATRIL ATRIO ATROZ ATUFA ATUFE ATUFO ATUSA ATUSE ATUSO ATUVE ATUVO AUDAZ AUDIO AUGES AUGUR AULAS AULLA AULLE AULLO AUNAD AUNAN AUNAR AUNAS AUNEN AUNES AUPAD AUPAN AUPAR AUPAS AUPEN AUPES AURAS AUREA AUREO AUTOR AUTOS AVALA AVALE AVALO AVARA AVARO AVECE AVENA AVENE AVENO AVEZA AVEZO AVIAD AVIAN AVIAR AVIAS AVICA AVIDA AVIDO AVIEN AVIES AVINE AVINO AVION AVISA AVISE AVISO AVIVA AVIVE AVIVO AVOCA AVOCO AVUGO AXIAL AXILA AYATE AYUDA AYUDE AYUDO AYUNA AYUNE AYUNO AZADA AZALA AZARA AZARE AZARO AZIMA AZIMO AZOAD AZOAN AZOAR AZOAS AZOCA AZOCO AZOEN AZOES AZOGA AZOGO AZOLE AZOLO AZORA AZORE AZORO AZOTA AZOTE AZOTO AZUAS AZUCE AZULA AZULE AZULO AZUZA AZUZO BABAS BABEA BABEE BABEL BABEO BABIA BABLE BABOR BACAS BACHE BACIA BACIN BACON BADAL BADAN BADEA BADEN BAFLE BAGAD BAGAN BAGAR BAGAS BAGOS BAGRE BAGUE BAHIA BAIDA BAILA BAILE BAILO BAJAD BAJAN BAJAR BAJAS BAJEL BAJEN BAJES BAJON BAJOS BALAD BALAN BALAR BALAS BALDA BALDE BALDO BALEA BALEN BALEO BALES BALIN BALON BALSA BALSO BALTA BALTO BAMBA BAMBU BANAL BANCA BANCE BANCO BANDA BANDO BANTU BANZO BAQUE BARBA BARBE BARBO BARCA BARCO BARDA BARDE BARDO BARES BARIA BARIO BARON BARRA BARRE BARRI BARRO BARZA BASAD BASAL BASAN BASAR BASAS BASCA BASEN BASES BASTA BASTE BASTO BATAN BATAS BATEA BATEE BATEL BATEN BATEO BATES BATIA BATID BATIN BATIO BATIR BATIS BATOS BAUZA BAYAL BAYAS BAYON BAYOS BAZAR BAZAS BAZOS BEATA BEATO BEBAN BEBAS BEBED BEBEN BEBER BEBES BEBIA BEBIO BECAD BECAN BECAR BECAS BEDEL BEFAD BEFAN BEFAR BEFAS BEFEN BEFES BEIGE BELDA BELDE BELDO BELEN BELEZ BELFA BELFO BELGA BELIO BELLA BELLO BEMOL BEODA BEODO BEQUE BERRO BERTA BERZA BESAD BESAN BESAR BESAS BESEN BESES BESOS BETAS BETUN BEZAR BICHA BICHO BICIS BIDES BIDON BIELA BIGAS BIJAS BILAO BILIS BIMBA BINAD BINAN BINAR BINAS BINEN BINES BINGO BINZA BIRLA BIRLE BIRLO BISEL BISOS BITER BIZCA BIZCO BLEDO BLOCA BLOCO BLUES BLUSA BOATO BOBAS BOBEA BOBEE BOBEO BOBOS BOCAL BOCAS BOCEA BOCEE BOCEO BOCHA BOCHE BOCHO BOCIN BOCIO BOCON BODAS BODON BOFES BOGAD BOGAN BOGAR BOGAS BOGUE BOHIO BOINA BOIRA BOJES BOLAR BOLAS BOLEA BOLEE BOLEO BOLES BOLIN BOLLA BOLLE BOLLO BOLOS BOLSA BOLSO BOMBA BOMBE BOMBO BONGA BONGO BONOS BONZO BORAX BORDA BORDE BORDO BOREA BOREO BORLA BORNE BOROS BORRA BORRE BORRO BOSTA BOTAD BOTAN BOTAR BOTAS BOTEN BOTES BOTIN BOTON BOXEA BOXEE BOXEO BOXER BOYAD BOYAL BOYAN BOYAR BOYAS BOYEN BOYES BOZAL BOZOS BRACO BRAGA BRAMA BRAME BRAMO BRASA BRAVA BRAVO BRAZA BRAZO BREAD BREAN BREAR BREAS BRECA BREEN BREES BREGA BREGO BRETE BREVA BREVE BREZO BRIDA BRIOS BRISA BRIZA BROCA BROCE BROMA BROME BROMO BROTA BROTE BROTO BROZA BROZO BRUGO BRUJA BRUJO BRUMA BRUTA BRUTO BUBAS BUBON BUCAL BUCEA BUCEE BUCEN BUCEO BUCES BUCHE BUCLE BUCOS BUDAS BUDIN BUENA BUENO BUFAD BUFAN BUFAR BUFAS BUFEN BUFES BUFON BUFOS BUHOS BUJES BUJIA BULAS BULBO BULLA BULLE BULLI BULLO BULOS BULTO BUQUE BURDA BURDO BUREO BURGA BURGO BURIL BURLA BURLE BURLO BUROS BURRA BURRO BUSCA BUSCO BUSES BUSTO BUYOS BUZAD BUZAN BUZAR BUZAS BUZON BUZOS BYTES CABAL CABAS CABED CABEN CABER CABES CABIA CABLE CABOS CABRA CABRE CACAO CACAS CACEA CACEE CACEN CACEO CACES CACHA CACHE CACHO CACOS CACTO CAEIS CAEME CAEOS CAERA CAERE CAETE CAFES CAFRE CAGAD CAGAN CAGAR CAGAS CAGON CAGUE CAIAN CAIAS CAICO CAIDA CAIDO CAIGA CAIGO CAJAS CAJIN CAJON CALAD CALAN CALAO CALAR CALAS CALCA CALCE CALCO CALDA CALDO CALEN CALES CALIZ CALLA CALLE CALLO CALMA CALME CALMO CALON CALOR CALOS CALVA CALVE CALVO CALZA CALZO CAMAS CAMBA CAMBO CAMON CAMPA CAMPE CAMPO CANAL CANAS CANDA CANDE CANDO CANEA CANEE CANEO CANES CANEY CANJE CANOA CANON CANOS CANSA CANSE CANSO CANTA CANTE CANTO CAOBA CAPAD CAPAN CAPAR CAPAS CAPAZ CAPEA CAPEE CAPEN CAPEO CAPES CAPIN CAPON CAPTA CAPTE CAPTO CAQUI CARAS CARAY CARCA CARDA CARDE CARDO CAREA CAREE CAREO CAREY CARGA CARGO CARIA CARIE CARIO CARIZ CARLA CARLO CARNE CAROS CARPA CARPO CARRO CARTA CASAD CASAL CASAN CASAR CASAS CASCA CASCO CASEA CASEN CASEO CASES CASIO CASON CASOS CASPA CASTA CASTO CATAD CATAN CATAR CATAS CATEA CATEE CATEN CATEO CATES CATIN CATON CATOS CATRE CAUCE CAUSA CAUSE CAUSO CAUTA CAUTO CAVAD CAVAN CAVAR CAVAS CAVEA CAVEN CAVES CAYOS CAZAD CAZAN CAZAR CAZAS CAZON CAZUZ CCITT CEBAD CEBAN CEBAR CEBAS CEBEN CEBES CEBON CEBOS CEBRA CECEA CECEE CECEO CEDAN CEDAS CEDED CEDEN CEDER CEDES CEDIA CEDIO CEDRO CEGAD CEGAR CEGUE CEJAD CEJAN CEJAR CEJAS CEJEN CEJES CEJOS CELAD CELAN CELAR CELAS CELDA CELEN CELES CELIA CELOS CELTA CENAD CENAN CENAR CENAS CENEN CENES CENIT CENSA CENSE CENSO CEPAS CEPOS CERAS CERCA CERCO CERDA CERDO CEREA CEREO CERIO CERNI CERON CEROS CERRE CERRO CESAD CESAN CESAR CESAS CESEN CESES CESIO CESTA CESTO CETRO CEUTI CHACO CHAFA CHAFE CHAFO CHALA CHALE CHALO CHAPA CHAPE CHAPO CHATA CHATO CHECA CHECO CHEFS CHEPA CHICA CHICO CHILE CHINA CHINE CHINO CHIPS CHIST CHITO CHIVA CHIVE CHIVO CHOCA CHOCO CHOLA CHOPO CHOTA CHOTO CHOVA CHOZA CHOZO CHUFA CHULA CHULO CHUPA CHUPE CHUPO CHUTA CHUTE CHUTO CHUZA CHUZO CIABA CIADA CIADO CIAIS CIARA CIARE CIASE CICLA CICLE CICLO CIDES CIDRA CIDRO CIEGA CIEGO CIEIS CIELO CIENO CIFRA CIFRE CIFRO CILIO CIMAS CINCA CINCO CINES CINTA CINTE CINTO CIRCE CIRCO CIRIO CIRRO CISCA CISCO CISMA CISME CISMO CISNE CITAD CITAN CITAR CITAS CITEN CITES CITOS CIVIL CLAMA CLAME CLAMO CLARA CLARO CLASE CLAVA CLAVE CLAVO CLERO CLIMA CLISE CLOCO CLONA CLONE CLONO CLORA CLORE CLORO COBAS COBOL COBRA COBRE COBRO COCAD COCAN COCAR COCAS COCEA COCED COCEE COCEO COCER COCES COCHE COCIA COCIO COCOS CODEA CODEE CODEO CODOS COFAS COFIA COFRE COGED COGEN COGER COGES COGIA COGIO COGON COITO COJAN COJAS COJEA COJEE COJEO COJIN COJOS COLAD COLAR COLAS COLEA COLEE COLEO COLES COLGO COLIN COLMA COLME COLMO COLON COLOR COLZA COMAN COMAS COMBA COMBE COMBO COMED COMEN COMER COMES COMIA COMIC COMIO COMUN CONDE CONGA CONGO CONOS CONTE CONTO COPAD COPAN COPAR COPAS COPEA COPEE COPEN COPEO COPES COPIA COPIE COPIO COPLA COPON COPOS COQUE CORAL CORAN CORDA COREA COREE COREO CORNO COROS CORRA CORRE CORRI CORRO CORSA CORSE CORSO CORTA CORTE CORTO CORVA CORVO CORZA CORZO COSAN COSAS COSCA COSCO COSED COSEN COSER COSES COSIA COSIO COSOS COSTA COSTE COSTO COTAD COTAN COTAR COTAS COTEN COTES COTON COTOS COXAL COXIS CRASA CRASO CREAD CREAN CREAR CREAS CRECE CRECI CREDO CREED CREEN CREER CREES CREIA CREMA CRETA CREYO CRIAD CRIAN CRIAR CRIAS CRIBA CRIBE CRIBO CRIEN CRIES CRIOS CROAD CROAN CROAR CROAS CROEN CROES CROMA CROME CROMO CRONO CRUCE CRUDA CRUDO CRUEL CRUJA CRUJE CRUJI CRUJO CRUOR CRUZA CRUZO CUAJA CUAJE CUAJO CUBAS CUBIL CUBOS CUBRA CUBRE CUBRI CUBRO CUCAD CUCAN CUCAR CUCAS CUCOS CUECE CUELA CUELE CUELO CUERA CUERO CUEVA CUEZA CUEZO CUIDA CUIDE CUIDO CUITA CULAR CULEA CULEE CULEO CULON CULOS CULPA CULPE CULPO CULTA CULTO CUNAD CUNAN CUNAR CUNAS CUNDA CUNDE CUNDI CUNDO CUNEA CUNEE CUNEN CUNEO CUNES CUOTA CUPES CUPLE CUPON CUPOS CUQUE CURAD CURAN CURAR CURAS CURDA CURDO CUREN CURES CURIA CURIO CURRA CURRE CURRO CURSA CURSE CURSI CURSO CURTA CURTE CURTI CURTO CURVA CURVE CURVO CUTER CUTIS CUTOS CUTRE CUYAS CUYOS CUZCO DABAN DABAS DABLE DACAS DACHA DACIA DACIO DADAS DADLA DADLE DADLO DADME DADOR DADOS DAGAS DALAS DALES DALIA DALLE DALOS DAMAS DAMOS DANCE DANDI DANDO DANES DANOS DANTE DANZA DANZO DARAN DARAS DARDO DARIA DARLA DARLE DARLO DARME DAROS DARSE DARTE DATAD DATAN DATAR DATAS DATEN DATES DATIL DATOS DAVID DEBAN DEBAS DEBED DEBEN DEBER DEBES DEBIA DEBIL DEBIO DEBUT DECAE DECAI DECIA DECID DECIR DECIS DEDAL DEDIL DEDOS DEJAD DEJAN DEJAR DEJAS DEJEN DEJES DELAS DELES DELGA DELIA DELOS DELTA DEMAS DEMOS DENLA DENLE DENLO DENME DENOS DENSA DENSE DENSO DENTA DENTE DENTO DEPON DERBI DESDA DESDE DESDO DESEA DESEE DESEO DESGA DESOI DETEN DEUDA DEUDO DEVEN DIADA DIADO DIANA DICAZ DICEN DICES DICHA DICHO DICTA DICTE DICTO DIEGO DIERA DIERE DIESE DIETA DIETE DIETO DIGAN DIGAS DIGNA DIGNE DIGNO DIJES DILAS DILES DILOS DILUI DIMES DIMOS DINAR DINOS DIODO DIOSA DIQUE DIRAN DIRAS DIRIA DISCO DISON DISTA DISTE DISTO DITAS DIUCA DIVAN DIVAS DIVOS DOBLA DOBLE DOBLO DOCAS DOCES DOCIL DOCTA DOCTO DOGAL DOGAS DOGMA DOGOS DOGRE DOLAD DOLAR DOLED DOLER DOLIA DOLIO DOLOR DOLOS DOMAD DOMAN DOMAR DOMAS DOMBO DOMEN DOMES DOMOS DONAD DONAN DONAR DONAS DONDE DONEN DONES DOPAD DOPAN DOPAR DOPAS DOPEN DOPES DORAD DORAL DORAN DORAR DORAS DOREN DORES DORIA DORIO DORMI DORNA DORSO DOSEL DOSES DOSIS DOTAD DOTAL DOTAN DOTAR DOTAS DOTEN DOTES DRABA DRAGA DRAGO DRAMA DRENA DRENE DRENO DRIAS DRINO DRIZA DROGA DROGO DROPE DRUPA DRUSA DRUSO DUBAS DUBIO DUCAL DUCES DUCHA DUCHE DUCHO DUDAD DUDAN DUDAR DUDAS DUDEN DUDES DUELA DUELE DUELO DUETO DUJOS DULAR DULAS DULCE DULIA DUMAS DUNAS DUNDA DUNDO DUPLA DUPLO DUQUE DURAD DURAN DURAR DURAS DUREN DURES DUROS EBANO EBRIA EBRIO ECHAD ECHAN ECHAR ECHAS ECHEN ECHES ECUAS ECUOS EDEMA EDILA EDITA EDITE EDITO EDRAD EDRAN EDRAR EDRAS EDREN EDRES EDUCA EDUCO EFEBO EFETA EFLUI EGIDA EIRAS EJIDO EJION ELATO ELCHE ELEGI ELEPE ELEVA ELEVE ELEVO ELFOS ELIDA ELIDE ELIDI ELIDO ELIGE ELIJA ELIJE ELIJO ELITE ELLAS ELLES ELLOS ELOTE ELUDA ELUDE ELUDI ELUDO EMANA EMANE EMANO EMITA EMITE EMITI EMITO EMULA EMULE EMULO ENANA ENANO ENCIA ENDES ENEAL ENEAS ENEJA ENEJE ENEJO ENEMA ENERO ENOJA ENOJE ENOJO ENRIA ENRIE ENRIO ENSAY ENTES ENTRA ENTRE ENTRO ENVES ENVIA ENVIE ENVIO ENZAS EOLIA EOLIO EONES EPATA EPATE EPATO EPICA EPICO EPOCA EPOXI EPROM EQUIS ERABA ERADA ERADO ERAIS ERAJE ERALA ERARA ERARE ERASE ERBIO EREBO EREIS ERGIO ERGOS ERGUI ERIAL ERIAS ERICE ERIGE ERIGI ERIJA ERIJO ERINA ERIOS ERIZA ERIZO EROGA EROGO ERRAD ERRAN ERRAR ERRAS ERREN ERRES ERROR ERUTA ERUTE ERUTO ESCOA ESMUI ESNOB ESPIA ESPIE ESPIN ESPIO ESQUI ESTAD ESTAN ESTAR ESTAS ESTEN ESTER ESTES ESTIO ESTOS ESTOY ESTRO ETANO ETAPA ETICA ETICO ETILO ETIMO ETNIA EUROS EVADA EVADE EVADI EVADO EVITA EVITE EVITO EVOCA EVOCO EXIGE EXIGI EXIJA EXIJO EXILA EXILE EXILO EXIMA EXIME EXIMI EXIMO EXITO EXODO EXORA EXORE EXORO EXPIA EXPIE EXPIO EXPON EXTRA EXUDA EXUDE EXUDO FABLA FACAS FACES FACHA FACIL FACON FADOS FAENA FAENE FAENO FAGOT FAJAD FAJAN FAJAR FAJAS FAJEN FAJES FAJIN FAJON FAJOS FALAZ FALCA FALDA FALLA FALLE FALLO FALOS FALSA FALSO FALTA FALTE FALTO FALUA FAMAS FANAL FANES FANGO FARDA FARDE FARDO FAROL FAROS FARRA FARSA FASES FASTA FASTO FATAL FATOS FATUA FATUO FAUNA FAUNO FAVOR FAXES FEBLE FECAL FECHA FECHE FECHO FELIX FELIZ FELON FELPA FELPE FELPO FEMUR FENDA FENIX FENOL FERAZ FERIA FERIE FERIO FEROE FEROZ FERRA FERRE FERRO FERRY FETAL FETCH FETEN FETOS FEUCA FEUCO FEUDA FEUDE FEUDO FIABA FIADA FIADO FIAIS FIALA FIALE FIALO FIAME FIAOS FIARA FIARE FIASE FIATE FIBRA FICHA FICHE FICHO FICUS FIDEO FIEIS FIELA FIELE FIELO FIEME FIERA FIERO FIESE FIGLE FIGON FIJAD FIJAN FIJAR FIJAS FIJEN FIJES FIJOS FILAD FILAN FILAR FILAS FILEN FILES FILFA FILIA FILIE FILIO FILMA FILME FILMO FILON FILOS FINAD FINAL FINAN FINAR FINAS FINCA FINCO FINEN FINES FINGE FINGI FINJA FINJO FINOS FINTA FINTE FINTO FIQUE FIRMA FIRME FIRMO FISCO FISGA FISGO FLACA FLACO FLAMA FLASH FLATO FLECO FLEJE FLEMA FLETA FLETE FLETO FLEXO FLOJA FLOJO FLORA FLORE FLORO FLOTA FLOTE FLOTO FLUIA FLUID FLUIR FLUIS FLUJO FLUOR FLUYA FLUYE FLUYO FOBIA FOCAL FOCAS FOCHA FOCOS FOFAS FOFOS FOGON FOJAS FOLIA FOLIE FOLIO FOLLA FOLLE FOLLO FONDA FONDO FONJE FORAL FORCE FORJA FORJE FORJO FORMA FORME FORMO FOROS FORRA FORRE FORRO FORZO FOSAD FOSAN FOSAR FOSAS FOSCA FOSCO FOSEN FOSES FOSIL FOSOS FOTON FOTOS FRAGA FRASE FRECE FREGO FREIA FREID FREIR FREIS FRENA FRENE FRENO FREON FREOS FRESA FRESE FRESO FREZA FREZO FRIAN FRIAS FRIEN FRIES FRIOS FRISA FRISE FRISO FRITA FRITE FRITO FROTA FROTE FROTO FRUTA FRUTE FRUTO FUCOS FUEGO FUERA FUERE FUERO FUESE FUGAD FUGAN FUGAR FUGAS FUGAZ FUGUE FUINA FULAR FULGE FULGI FULJA FULJO FUMAD FUMAN FUMAR FUMAS FUMEN FUMES FUNDA FUNDE FUNDI FUNDO FURIA FUROR FUSAS FUSCA FUSIL FUSOR FUSTA FUSTE FUTIL FUTRE GABAN GACEL GACHA GACHI GACHO GAFAD GAFAN GAFAR GAFAS GAFEN GAFES GAFOS GAGAS GAGOS GAITA GAJES GAJOS GALAN GALAS GALES GALGA GALGO GALIO GALLO GALON GALOS GAMAS GAMBA GAMMA GAMON GAMOS GANAD GANAN GANAR GANAS GANEN GANES GANGA GANSA GANSO GANTE GARBA GARBO GARFA GARLA GARRA GARUA GARUE GARUO GARZA GARZO GASAS GASEA GASEE GASEO GASES GASON GASTA GASTE GASTO GATAS GATEA GATEE GATEO GATOS GAUSS GAVIA GAYAS GAYOS GAZAS GBYTE GELES GEMAS GEMIA GEMID GEMIR GEMIS GENES GENIA GENIO GENTE GESTA GESTE GESTO GFLOP GHCES GIBAD GIBAN GIBAR GIBAS GIBEN GIBES GIGAS GILES GIMAN GIMAS GIMEN GIMES GIMIO GIRAD GIRAN GIRAR GIRAS GIREN GIRES GIROS GISES GLASE GLEBA GLIAL GLIAS GLIFO GLOBO GLOSA GLOSE GLOSO GNEIS GNOMO GOBIO GOCEN GOCES GOCHA GOCHO GODAS GODOS GOFIO GOFRA GOFRE GOFRO GOLAS GOLEA GOLEE GOLEO GOLES GOLFA GOLFO GOLPE GOMAS GOMER GORDA GORDO GORJA GORRA GORRO GOTAS GOTEA GOTEE GOTEO GOTON GOZAD GOZAN GOZAR GOZAS GOZNE GOZOS GRABA GRABE GRABO GRADA GRADE GRADO GRAFO GRAJA GRAJO GRAMA GRAMO GRANA GRANE GRANO GRAOS GRAPA GRAPE GRAPO GRASA GRASO GRATA GRATO GRAVA GRAVE GRAVO GREBA GRECA GRECO GREDA GREEN GRELO GRIAL GRIFA GRIFO GRILL GRIMA GRIPE GRISU GRITA GRITE GRITO GROSO GRUAS GRUMO GRUPA GRUPO GRUTA GUACA GUACO GUAJE GUAMA GUANO GUAPA GUAPO GUARA GUARO GUASA GUASO GUATA GUATE GUAYA GUAYE GUAYO GUBIA GUETO GUIAD GUIAN GUIAR GUIAS GUIDO GUIEN GUIES GUIJA GUIJO GUION GUIPA GUIPE GUIPO GUIRA GUIRI GUISA GUISE GUISO GUITA GUITE GUITO GULAG GULAS GUMIA GURDA GURDO GUROS GUSTA GUSTE GUSTO GUZLA HABAR HABAS HABER HABIA HABIL HABIZ HABLA HABLE HABLO HABON HABRA HABRE HABUS HACAN HACAS HACED HACEN HACER HACES HACHA HACHE HACHO HACIA HADAS HADOS HAFIZ HAGAN HAGAS HALAD HALAN HALAR HALAS HALDA HALEN HALES HALLA HALLE HALLO HALON HALOS HAMEZ HAMPA HANZO HARAN HARAS HARBA HARBE HARBO HARCA HAREM HAREN HARIA HARON HARPA HARRE HARTA HARTE HARTO HASTA HATEA HATEE HATEO HATOS HAUTE HAVAR HAVOS HAYAL HAYAN HAYAS HAYOS HAZAS HAZLA HAZLE HAZLO HAZME HAZTE HEBEN HEBRA HECES HECHA HECHO HECTO HEDED HEDER HEDIA HEDIO HEDOR HELAD HELAR HELEA HELEE HELEO HELIO HELOR HEMOS HENAL HENAR HENDA HENDE HENDI HENDO HENIL HENOS HERBE HERBO HERIA HERID HERIL HERIR HERIS HERMA HEROE HERPE HERRE HERRO HERVI HESPA HESPE HESPI HESPO HETEA HETEO HIATO HIDRA HIDRO HIEDA HIEDE HIEDO HIELA HIELE HIELO HIENA HIERA HIERE HIERO HIGAS HIGOS HIJAS HIJOS HILAD HILAN HILAR HILAS HILEN HILES HILOS HIMEN HIMNO HINCA HINCO HINDU HINES HIPAD HIPAN HIPAR HIPAS HIPEN HIPER HIPES HIPOS HIRCO HIRIO HIRMA HIRME HIRMO HISCA HISPA HISPE HISPI HISPO HITAD HITAN HITAR HITAS HITEN HITES HITON HITOS HOBOS HOCEN HOCES HOGAR HOJAS HOJEA HOJEE HOJEO HOLAN HOLAS HOLCO HOLEA HOLEE HOLEO HOLGO HOLLE HOLLO HONDA HONDO HONGO HONOR HONRA HONRE HONRO HOPAS HOPEA HOPEE HOPEO HOPOS HOQUE HORAS HORCA HORCO HORDA HORMA HORNO HORRA HORRE HORRO HOSCA HOSCO HOSPA HOTEL HOTOS HOVES HOYAS HOYOS HOZAD HOZAN HOZAR HOZAS HUACA HUACO HUCHA HUCHO HUECA HUECO HUELA HUELE HUELO HUERA HUERO HUESA HUESO HUEVA HUEVE HUEVO HUIAN HUIAS HUICH HUIDA HUIDO HUIFA HUIRA HUIRE HUIRO HULES HULLA HUMAD HUMAN HUMAR HUMAS HUMEA HUMEE HUMEN HUMEO HUMES HUMOR HUMOS HUMUS HUNAS HUNDA HUNDE HUNDI HUNDO HUNOS HUPES HURAS HURGA HURGO HURON HURRA HURTA HURTE HURTO HUSAR HUSMA HUSMO HUSOS HUTAS HUTIA HUYAN HUYAS HUYEN HUYES IBAIS IBERA IBERO IBICE ICACO ICEIS ICELA ICELE ICELO ICEME ICESE ICHAL ICHOS ICONO ICTUS IDEAD IDEAL IDEAN IDEAR IDEAS IDEEN IDEES IDEOS IDOLA IDOLO IGNEA IGNEO IGUAL IJADA IJUJU ILEAS ILEON ILEOS ILESA ILESO ILOTA ILUDA ILUDE ILUDI ILUDO ILUSA ILUSO IMADA IMANA IMANE IMANO IMBUI IMELA IMITA IMITE IMITO IMPAR IMPIA IMPIO IMPLA IMPON INANE INCAS INCOA INCOE INCOO INDAS INDIA INDIO INDOS INFLA INFLE INFLO INGLE INPUT INSTA INSTE INSTO INTER INTUI IONES IOTAS IRANI IREIS IRENE IRIAN IRIAS IRISA IRISE IRISO IRNOS ISAAC ISBAS ISLAM ISLAS ISTMO ITALA ITALO ITERA ITERE ITERO ITRIA ITRIO IZABA IZADA IZADO IZAIS IZALA IZALE IZALO IZAME IZAOS IZARA IZARE IZASE IZATE IZOTE JABAS JABON JACAS JACOS JACTA JACTE JACTO JADEA JADEE JADEO JADES JAECE JAEZA JAEZO JAIBA JAIMA JAIME JALAD JALAN JALAR JALAS JALDA JALDE JALDO JALEA JALEE JALEN JALEO JALES JALMA JALON JAMAD JAMAN JAMAR JAMAS JAMBA JAMEN JAMES JAMON JAPON JAQUE JARAL JARAS JAROS JARRA JARRE JARRO JASAD JASAN JASAR JASAS JASEN JASES JASPE JATAS JATEA JATEO JATIB JATOS JAUDA JAUDO JAUJA JAULA JAUTA JAUTO JAVAS JAVOS JAYAN JEBES JEDAD JEDAN JEDAR JEDAS JEDEN JEDES JEFAS JEFES JEITO JEJEN JELIZ JEMAL JEMES JEQUE JERAS JERBO JEREZ JERGA JERPA JESUS JETAD JETAN JETAR JETAS JETEN JETES JETON JETOS JIBIA JIFAS JIFIA JIGAS JIJAS JIJEA JIJEE JIJEO JIMIA JIMIO JIRAS JIRON JISCA JITAD JITAN JITAR JITAS JITEN JITES JOBOS JOCOS JODAN JODAS JODED JODEN JODER JODES JODIA JODIO JOFOR JONDO JONIA JONIO JOPAD JOPAN JOPAR JOPAS JOPEN JOPES JOPOS JORAS JORCO JORFE JORGE JORRA JORRE JORRO JOSAS JOTAS JOTES JOULE JOVEN JOYAS JOYON JOYOS JUBAS JUBON JUBOS JUDAS JUDEA JUDIA JUDIO JUDIT JUEGA JUEGO JUERA JUEZA JUGAD JUGAR JUGOS JUGUE JUJEO JULIA JULIO JULOS JUMAS JUNCO JUNIO JUNTA JUNTE JUNTO JUNZA JURAD JURAN JURAR JURAS JUREL JUREN JURES JUROS JUSTA JUSTE JUSTO JUTAS JUTIA JUVIA JUZGA JUZGO KANES KAPPA KARMA KARST KBITS KBYTE KEFIR KHCES KILOS KIRIE KIVIS KIWIS KOPEK KURDA KURDO LABIA LABIL LABIO LABOR LABRA LABRE LABRO LACAD LACAN LACAR LACAS LACEA LACEE LACEN LACEO LACES LACHA LACIA LACIO LACON LACRA LACRE LACRO LACTA LACTE LACTO LADAS LADEA LADEE LADEO LADON LADOS LADRA LADRE LADRO LAGAR LAGOS LAICA LAICO LAJAS LAMAN LAMAS LAMED LAMEN LAMER LAMES LAMIA LAMIN LAMIO LAMPA LAMPE LAMPO LANAR LANAS LANCE LANDA LANDO LANGA LANIA LANIO LANZA LANZO LAPAS LAPIZ LAPON LAPOS LAPSO LAQUE LARDA LARDE LARDO LARES LARGA LARGO LARRA LARVA LASAS LASCA LASCO LASER LASOS LASTA LASTE LASTO LASUN LATAN LATAS LATAZ LATEN LATES LATEX LATIA LATID LATIN LATIO LATIR LATIS LATON LATOS LAUDA LAUDE LAUDO LAUNA LAURA LAURO LAVAD LAVAN LAVAR LAVAS LAVEN LAVES LAXAD LAXAN LAXAR LAXAS LAXEN LAXES LAXOS LAYAD LAYAN LAYAR LAYAS LAYEN LAYES LAZAD LAZAN LAZAR LAZAS LAZOS LEAIS LEALA LEALE LEALO LEAME LEASE LECHA LECHE LECHO LEDAS LEDOS LEEIS LEELA LEELE LEELO LEEME LEEOS LEERA LEERE LEETE LEGAD LEGAL LEGAN LEGAR LEGAS LEGON LEGOS LEGRA LEGRE LEGRO LEGUA LEGUE LEIAN LEIAS LEIDA LEIDO LEILA LEIMA LEJAS LEJIA LEJOS LELAS LELOS LEMAS LEMUR LENAS LENES LENTA LENTE LENTO LEONA LEPRA LERAS LERDA LERDO LESAS LESNA LESOS LESTE LETAL LETEA LETEO LETON LETRA LEUDA LEUDE LEUDO LEVAD LEVAN LEVAR LEVAS LEVEN LEVES LEYES LEZDA LEZNA LIABA LIADA LIADO LIAIS LIALA LIALE LIALO LIAME LIANA LIAOS LIARA LIARE LIASE LIATE LIAZA LIBAD LIBAN LIBAR LIBAS LIBEN LIBER LIBES LIBIA LIBIO LIBON LIBRA LIBRE LIBRO LICEO LICIA LICIO LICOR LICUA LICUE LICUO LIDER LIDES LIDIA LIDIE LIDIO LIEGA LIEGO LIEIS LIELA LIELE LIELO LIEME LIESE LIGAD LIGAN LIGAR LIGAS LIGIO LIGON LIGUE LIGUR LIJAD LIJAN LIJAR LIJAS LIJEN LIJES LILAC LILAO LILAS LIMAD LIMAN LIMAR LIMAS LIMBO LIMEN LIMES LIMON LIMOS LINAO LINAR LINCE LINDA LINDE LINDO LINEA LINEE LINEO LINFA LINIO LINON LINOS LINUX LIOSA LIOSO LIPES LIPIS LIRAS LIRIA LIRIO LIRON LISAS LISIA LISIE LISIO LISIS LISOS LISTA LISTE LISTO LITAD LITAN LITAR LITAS LITEN LITES LITIO LITIS LITRE LITRO LITUO LIVOR LIZAS LIZOS LLACA LLAGA LLAGO LLAMA LLAME LLAMO LLANA LLANO LLAPA LLAPE LLAPO LLAVE LLECA LLECO LLEGA LLEGO LLENA LLENE LLENO LLERA LLEVA LLEVE LLEVO LLORA LLORE LLORO LLOSA LLOVI LOABA LOADA LOADO LOAIS LOARA LOARE LOASE LOBAS LOBEA LOBEE LOBEO LOBOS LOCAL LOCAS LOCHA LOCHE LOCOS LOCRO LODON LODOS LOEIS LOGIA LOGIN LOGIS LOGOS LOGRA LOGRE LOGRO LOICA LOINA LOMAS LOMBA LOMEA LOMEE LOMEO LOMOS LONAS LONJA LORAS LORES LOROS LORZA LOSAD LOSAN LOSAR LOSAS LOSEN LOSES LOTAS LOTES LOTOS LOZAS LUCEN LUCES LUCHA LUCHE LUCHO LUCIA LUCID LUCIO LUCIR LUCIS LUCRA LUCRE LUCRO LUDAN LUDAS LUDEN LUDES LUDIA LUDID LUDIE LUDIO LUDIR LUDIS LUEGO LUGAR LUGRE LUIAN LUIAS LUIDA LUIDO LUIRA LUIRE LUISA LUJOS LUMAS LUMIA LUNAR LUNAS LUNES LUPAS LUPIA LUPUS LURTE LUSAS LUSOS LUTEA LUTEO LUTOS LUXAD LUXAN LUXAR LUXAS LUXEN LUXES LUYAN LUYAS LUYEN LUYES LUZCA LUZCO LYCRA MACAD MACAN MACAR MACAS MACEA MACEE MACEN MACEO MACES MACHA MACHE MACHI MACHO MACIA MACIS MACON MACOS MACRO MADOR MADRE MAFIA MAGAS MAGIA MAGIN MAGMA MAGNA MAGNO MAGOS MAGRA MAGRO MAHON MAIDO MAJAD MAJAL MAJAN MAJAR MAJAS MAJEN MAJES MAJOS MALAR MALAS MALEA MALEE MALEO MALES MALLA MALLE MALLO MALON MALOS MALTA MALVA MAMAD MAMAN MAMAR MAMAS MAMBI MAMBO MAMEN MAMES MAMEY MAMIA MAMON MAMUT MANAD MANAN MANAR MANAS MANCA MANCO MANDA MANDE MANDI MANDO MANEA MANEE MANEN MANEO MANES MANGA MANGO MANIA MANID MANIO MANIR MANIS MANOS MANSA MANSO MANTA MANTO MAORI MAPAS MAPEA MAPEE MAPEO MAQUE MAQUI MARCA MARCO MAREA MAREE MAREO MARES MARGA MARGO MARIO MARON MAROS MARRA MARRE MARRO MARSA MARSO MARTA MARTE MARZO MASAD MASAN MASAR MASAS MASCA MASCO MASEN MASES MASIA MASLO MASON MASTO MATAD MATAN MATAR MATAS MATEA MATEE MATEN MATEO MATES MATIZ MATON MATOS MAULA MAULE MAULO MAURA MAURE MAURO MAYAL MAYAS MAYEA MAYEE MAYEO MAYOR MAYOS MAZAD MAZAN MAZAR MAZAS MAZNA MAZNE MAZNO MAZOS MBITS MBYTE MEABA MEADA MEADO MEAIS MEAJA MEALA MEALE MEALO MEAME MEAOS MEARA MEARE MEASE MEATE MEATO MECED MECEN MECER MECES MECHA MECHE MECHO MECIA MECIO MECOS MEDAS MEDIA MEDID MEDIE MEDIO MEDIR MEDIS MEDOS MEDRA MEDRE MEDRO MEEIS MEELA MEELE MEELO MEEME MEESE MEGAS MEGOS MEIGA MEIGO MEJOR MELAD MELAR MELAS MELCA MELGA MELGO MELIS MELLA MELLE MELLO MELON MELSA MELVA MEMAS MEMEZ MEMOS MENAD MENAN MENAR MENAS MENDA MENEA MENEE MENEN MENEO MENES MENOR MENOS MENTA MENTE MENTI MENTO MENUS MEONA MERAD MERAN MERAR MERAS MERCA MERCO MEREN MERES MEREY MERGO MERLA MERLO MERMA MERME MERMO MEROS MESAD MESAN MESAR MESAS MESEN MESES MESON MESTA MESTO METAL METAN METAS METED METEN METER METES METIA METIO METRA METRO MEZAN MEZAS MFLOP MHCES MIABA MIADA MIADO MIAGA MIAGO MIAIS MIAJA MIARA MIARE MIASE MICAS MICER MICES MICHA MICHO MICOS MICRA MICRO MIDAN MIDAS MIDEN MIDES MIDIO MIEDO MIEIS MIELA MIELE MIELO MIERA MIGAD MIGAN MIGAR MIGAS MIGRA MIGRE MIGRO MIGUE MIJOS MILAN MILES MILLA MILLO MILPA MIMAD MIMAN MIMAR MIMAS MIMEN MIMES MIMOS MINAD MINAL MINAN MINAR MINAS MINEN MINES MINGA MINGO MINIA MINIE MINIO MINIX MINOS MINUE MIOMA MIOPE MIRAD MIRAN MIRAR MIRAS MIREN MIRES MIRIA MIRLA MIRLE MIRLO MIRON MIRRA MIRTO MIRZA MISAD MISAL MISAN MISAR MISAS MISEN MISES MISIA MISIL MISIO MISMA MISMO MISTA MISTE MISTO MITAD MITAN MITAS MITIN MITON MITOS MITRA MITRE MITRO MIXTA MIXTO MIZAS MIZOS MOARE MOCAD MOCAN MOCAR MOCAS MOCEA MOCEE MOCEO MOCHA MOCHE MOCHO MOCIL MOCOS MODAL MODAS MODEM MODIO MODOS MOFAD MOFAN MOFAR MOFAS MOFEN MOFES MOGOL MOGON MOHIN MOHOS MOHUR MOJAD MOJAN MOJAR MOJAS MOJEN MOJES MOJON MOJOS MOLAR MOLAS MOLDA MOLDE MOLDO MOLED MOLER MOLES MOLIA MOLIO MOLLA MOLLE MOLON MOLOS MOLSA MOLSO MOMEA MOMEE MOMEO MOMIA MOMIO MOMOS MONAS MONDA MONDE MONDO MONEA MONEE MONEO MONFI MONGO MONJA MONJE MONOS MONTA MONTE MONTO MOPAS MOQUE MORAD MORAL MORAN MORAR MORAS MORBO MORCA MORCO MORDI MOREA MOREN MOREO MORES MORFA MORGA MORIA MORID MORIR MORIS MORON MOROS MORRA MORRO MORSA MORSE MOSCA MOSCO MOSEN MOSTE MOSTO MOTAS MOTEA MOTEE MOTEL MOTEO MOTES MOTIF MOTIN MOTON MOTOR MOTOS MOVED MOVER MOVIA MOVIL MOVIO MOXAS MOXTE MOYAS MOYOS MOZAS MOZOS MUAIS MUARE MUCHA MUCHO MUDAD MUDAN MUDAR MUDAS MUDEN MUDES MUDEZ MUDOS MUECA MUEIS MUELA MUELE MUELO MUERA MUERE MUERO MUESO MUEVA MUEVE MUEVO MUFLA MUFLE MUFLI MUFLO MUFTI MUGAD MUGAN MUGAR MUGAS MUGEN MUGES MUGIA MUGID MUGIO MUGIR MUGIS MUGRE MUGUE MUIAN MUIAS MUIDA MUIDO MUJAN MUJAS MUJER MUJOL MULAR MULAS MULEO MULLA MULLE MULLI MULLO MULOS MULSA MULSO MULTA MULTE MULTO MUNDO MURAD MURAL MURAN MURAR MURAS MUREN MURES MURGA MURIA MURIO MUROS MURTA MUSAN MUSAS MUSCA MUSCO MUSEN MUSEO MUSES MUSGA MUSGO MUSIA MUSID MUSIO MUSIR MUSIS MUSLO MUTAD MUTAN MUTAR MUTAS MUTEN MUTES MUTIS MUTUA MUTUO NABAB NABAL NABAR NABAS NABLA NABOS NACAR NACED NACEN NACER NACES NACHA NACHO NACIA NACIO NACOS NADAD NADAN NADAR NADAS NADEN NADES NADIE NADIR NAFRA NAFRE NAFRO NAFTA NAGUA NAHUA NAIFE NAIPE NAIRE NAJAS NALCA NALGA NANAS NANCE NANEA NANEE NANEO NANSA NANSU NANTA NANTE NANTO NAPAS NAPEA NAPEO NAQUE NARDO NARES NARIZ NARRA NARRE NARRO NASAL NASAS NASON NASOS NATAL NATAS NATIA NATIO NATOS NAUTA NAVAL NAVAS NAVES NAVIO NAZCA NAZCO NAZIS NEBLI NECEA NECEE NECEO NECIA NECIO NEGAD NEGAR NEGRA NEGRO NEGUE NEGUS NEJAS NELDO NEMAS NEMEA NEMEO NEMES NENAS NENES NENIA NERON NESGA NESGO NETAS NETOS NEUMA NEURA NEVAD NEVAN NEVAR NEVAS NEVEN NEVES NEVUS NEXOS NIARA NIAZO NICHO NICLE NIDAL NIDIA NIDIO NIDOS NIEGA NIEGO NIELA NIELE NIELO NIETA NIETO NIEVE NIMBA NIMBE NIMBO NIMIA NIMIO NINFA NINFO NIOTO NIPAS NIPON NITOS NITRA NITRE NITRO NIVEA NIVEL NIVEO NOBEL NOBLE NOCAS NOCHE NOCLA NODOS NOGAL NOMAS NOMON NOMOS NONAS NONES NONIO NONOS NOPAL NORIA NORMA NORTE NOTAD NOTAN NOTAR NOTAS NOTEN NOTES NOTOS NOTRO NOVAD NOVAL NOVAN NOVAR NOVAS NOVEL NOVEN NOVES NOVIA NOVIO NOXAS NOYOS NUBES NUBIA NUBIL NUBIO NUBLA NUBLE NUBLO NUCAS NUCOS NUDAS NUDOS NUERA NUEVA NUEVE NUEVO NULAS NULOS NUMEN NUMOS NUNCA NURIA NUTRA NUTRE NUTRI NUTRO OASIS OBESA OBESO OBICE OBITO OBLEA OBOES OBOLO OBRAD OBRAN OBRAR OBRAS OBREN OBRES OBSTA OBSTE OBSTO OBTEN OBUES OBVIA OBVIE OBVIO OCASO OCELO OCENA OCHOS OCIAD OCIAN OCIAR OCIAS OCIEN OCIES OCIOS OCLES OCLUI OCRES OCTAL OCUJE OCUMO OCUPA OCUPE OCUPO ODEON ODIAD ODIAN ODIAR ODIAS ODIEN ODIES ODIOS ODRES OESTE OFITA OGROS OHMIO OIAIS OIBLE OIDAS OIDIO OIDLA OIDLE OIDLO OIDME OIDOR OIDOS OIGAN OIGAS OIMOS OIRAN OIRAS OIRIA OIRLA OIRLE OIRLO OIRME OIROS OIRSE OIRTE OISLO OISTE OJALA OJALE OJALO OJEAD OJEAN OJEAR OJEAS OJEEN OJEES OJEOS OJERA OJETE OJIVA OJOSA OJOSO OJOTA OLAIS OLAJE OLEAD OLEAN OLEAR OLEAS OLEEN OLEES OLEIS OLEOS OLERA OLERE OLIAN OLIAS OLIDA OLIDO OLIVA OLIVO OLLAR OLLAS OLMOS OMEGA OMEYA OMINA OMINE OMINO OMISA OMISO OMITA OMITE OMITI OMITO ONDAS ONDEA ONDEE ONDEO ONICE ONZAS OPACA OPACO OPALO OPERA OPERE OPERO OPINA OPINE OPINO OPIOS OPONE OPTAD OPTAN OPTAR OPTAS OPTEN OPTES OPUSE OPUSO ORABA ORADA ORADO ORAIS ORALA ORALE ORALO ORARA ORARE ORASE ORATE ORBES ORCAS ORCEN ORCES ORCOS ORDEN OREAD OREAN OREAR OREAS OREEN OREES OREIS OREJA ORELA ORELE ORELO ORFOS ORFRE ORGIA ORIBE ORINA ORINE ORINO ORION ORIVE ORLAD ORLAN ORLAR ORLAS ORLEN ORLES ORLOS ORNAR OROYA ORTOS ORUGA ORUJO ORZAD ORZAN ORZAR ORZAS OSABA OSADA OSADO OSAIS OSALA OSALE OSALO OSARA OSARE OSASE OSEAD OSEAN OSEAR OSEAS OSEEN OSEES OSEIS OSELA OSELE OSELO OSEOS OSERA OSERO OSMIO OSTIA OSTRA OSTRO OSUDA OSUDO OSUNA OSUNO OTEAD OTEAN OTEAR OTEAS OTEEN OTEES OTERO OTILA OTILE OTILO OTOBA OTRAS OTROS OVABA OVADA OVADO OVAIS OVALA OVALE OVALO OVARA OVARE OVASE OVEIS OVEJA OVERA OVERO OVIDO OVINA OVINO OVOLO OVULA OVULE OVULO OXIDA OXIDE OXIDO OXTES OYELA OYELE OYELO OYEME OYERA OYERE OYESE OYETE OZONA OZONO PABLO PACED PACEN PACER PACES PACIA PACIO PACON PACTA PACTE PACTO PADRE PAFIA PAFIO PAGAD PAGAN PAGAR PAGAS PAGOS PAGUE PAICO PAIRO PAJAR PAJAS PAJEA PAJEE PAJEO PAJES PAJIL PAJON PAJOS PALAS PALCO PALEA PALEE PALEO PALIA PALIE PALIO PALMA PALMO PALON PALOS PALPA PALPE PALPO PAMPA PANAL PANAS PANCO PANDA PANEL PANES PANJI PANSA PANZA PAPAD PAPAL PAPAN PAPAR PAPAS PAPEA PAPEE PAPEL PAPEN PAPEO PAPES PARAD PARAN PARAR PARAS PARCA PARCO PARDA PARDO PAREA PARED PAREE PAREN PAREO PARES PARGO PARIA PARID PARIO PARIR PARIS PARLA PARLE PARLO PARNE PARON PAROS PARRA PARTA PARTE PARTI PARTO PARVA PARVO PASAD PASAN PASAR PASAS PASEA PASEE PASEN PASEO PASES PASMA PASME PASMO PASOS PASTA PASTE PASTO PATAN PATAO PATAS PATEA PATEE PATEO PATER PATES PATIN PATIO PATON PATOS PAUSA PAUSE PAUSO PAUTA PAUTE PAUTO PAVAS PAVES PAVIA PAVON PAVOR PAVOS PAYAS PAYES PAYOS PAZAN PAZAS PAZOS PEAIS PEAJE PEANA PECAD PECAN PECAR PECAS PECES PECHA PECHE PECHO PECIO PEDAL PEDIA PEDID PEDIR PEDIS PEDOS PEDRO PEEIS PEEOS PEERA PEERE PEGAD PEGAN PEGAR PEGAS PEGON PEGUE PEIAN PEIAS PEIDA PEIDO PEINA PEINE PEINO PEJES PEJIN PELAD PELAN PELAR PELAS PELEA PELEE PELEN PELEO PELES PELLA PELLO PELMA PELON PELOS PELTA PENAD PENAL PENAN PENAR PENAS PENCA PENCO PENDA PENDE PENDI PENDO PENEN PENES PENOS PENSE PENSO PEPAS PEPES PEPON PEQUE PERAL PERAS PERCA PERDI PERLA PERLE PERLO PERNO PEROL PERRA PERRO PERSA PERTA PESAD PESAN PESAR PESAS PESCA PESCO PESEN PESES PESOS PESTE PETAD PETAN PETAR PETAS PETEN PETES PETOS PETRA PEZON PIABA PIADA PIADO PIAFA PIAFE PIAFO PIAIS PIANO PIARA PIARE PIASE PICAD PICAN PICAR PICAS PICHI PICIO PICON PICOR PICOS PIDAN PIDAS PIDEN PIDES PIDIO PIDON PIEIS PIEJO PIEZA PIFIA PIFIE PIFIO PILAR PILAS PILEO PILLA PILLE PILLO PILON PILOS PINAR PINCE PINES PINGA PINGO PINOS PINTA PINTE PINTO PINZA PINZO PIOJO PIOLA PIOLE PIOLO PIONA PIPAD PIPAN PIPAR PIPAS PIPEN PIPES PIPIA PIPIE PIPIO PIPOS PIQUE PIRAD PIRAN PIRAR PIRAS PIRCA PIRCO PIREN PIRES PIRON PIRRA PIRRE PIRRO PISAD PISAN PISAR PISAS PISCO PISEN PISES PISON PISOS PISPA PISTA PISTE PISTO PITAD PITAN PITAO PITAR PITAS PITEN PITES PITIA PITIO PITON PITOS PIULA PIULE PIULO PIUNE PIURE PIVOT PIXEL PIZCA PIZCO PIZZA PLACA PLACO PLAGA PLAGO PLANA PLANO PLATA PLATO PLAYA PLAZA PLAZO PLEBE PLECA PLEGO PLENA PLENO PLEON PLEXO PLICA PLISA PLISE PLISO PLOMA PLOME PLOMO PLUMA POBLE POBLO POBOS POBRE POCAS POCHA POCHO POCOS PODAD PODAN PODAR PODAS PODED PODEN PODER PODES PODIA PODIO PODON PODRA PODRE POEMA POETA POINO POLAR POLCA POLCO POLEA POLEN POLEO POLIN POLIO POLLA POLLO POLOS POLVO POMAR POMAS POMEZ POMOS POMPA POMPO PONCI PONED PONEN PONER PONES PONGA PONGO PONIA PONIS PONLA PONLE PONLO PONME PONTE PONTO POPAD POPAN POPAR POPAS POPEN POPES PORGA PORGO PORNO POROS PORRA PORRO PORTA PORTE PORTO POSAD POSAN POSAR POSAS POSCA POSEA POSEE POSEI POSEN POSEO POSES POSMA POSON POSOS POSTA POSTE POTAD POTAN POTAR POTAS POTEN POTES POTRA POTRO POYOS POZAL POZAS POZOS PRADO PRAVA PRAVO PRESA PRESO PREVE PREVI PRIMA PRIME PRIMO PRION PRIOR PRISA PRIVA PRIVE PRIVO PROAS PROBA PROBE PROBO PROIZ PROLE PRONO PRORA PROSA PRUNO PUABA PUADA PUADO PUAIS PUARA PUARE PUASE PUBER PUBES PUBIS PUCHO PUCIA PUDIO PUDOR PUDRA PUDRE PUDRI PUDRO PUEDA PUEDE PUEDO PUEIS PUFOS PUGAS PUGIL PUGNA PUGNE PUGNO PUJAD PUJAN PUJAR PUJAS PUJEN PUJES PUJOS PULAN PULAS PULEN PULES PULGA PULIA PULID PULIO PULIR PULIS PULLA PULPA PULPO PULSA PULSE PULSO PUMAS PUNAN PUNAS PUNCE PUNEN PUNES PUNGE PUNGI PUNIA PUNID PUNIO PUNIR PUNIS PUNJA PUNJO PUNTA PUNTE PUNTO PUNZA PUNZO PUPAS PURAS PUREA PUREE PUREO PURES PURGA PURGO PUROS PUTAS PUTEA PUTEE PUTEO PUTOS PUYAS PUYOS QUARK QUEDA QUEDE QUEDO QUEJA QUEJE QUEJO QUEMA QUEME QUEMI QUEMO QUENA QUEPA QUEPO QUERA QUESO QUIAS QUIEN QUIJO QUILA QUILO QUIMA QUIMO QUINA QUINO QUIOS QUISE QUISO QUITA QUITE QUITO QUIZA RABAS RABEA RABEE RABEL RABEO RABIA RABIE RABIO RABON RABOS RACHA RACHE RACHO RADAR RADAS RADIA RADIE RADIO RADIX RADON RAEIS RAERA RAERE RAFAS RAFEA RAFEE RAFEO RAFES RAFIA RAGUA RAHEZ RAIAN RAIAS RAIDA RAIDO RAIGA RAIGO RAIJO RAJAD RAJAN RAJAR RAJAS RAJEN RAJES RALAS RALEA RALEE RALEO RALLA RALLE RALLO RALLY RALOS RAMAL RAMAS RAMEA RAMEO RAMIO RAMON RAMOS RAMPA RAMPE RAMPO RANAS RANGO RANOS RAPAD RAPAN RAPAR RAPAS RAPAZ RAPEN RAPES RAPOS RAPTA RAPTE RAPTO RARAS RAREA RAREE RAREO RAROS RASAD RASAN RASAR RASAS RASCA RASCO RASEN RASES RASGA RASGO RASOS RASPA RASPE RASPO RATAS RATEA RATEE RATEO RATIO RATON RATOS RAUDA RAUDO RAULI RAUTA RAYAD RAYAN RAYAR RAYAS RAYEN RAYES RAYON RAYOS RAZAS RAZON REAMA REAME REAMO REARA REARE REARO REATA REATE REATO REBLA REBLE REBLO RECAE RECAI RECEN RECES RECIA RECIO RECRE RECTA RECTE RECTO RECUA REDES REDIL REDRO REGAD REGAR REGIA REGID REGIO REGIR REGIS REGLA REGLE REGLO REGUE REHAZ REHEN REHUI REIAN REIAS REIDA REIDO REINA REINE REINO REIOS REIRA REIRE REJAS REJON REJOS RELAX RELEA RELEE RELEI RELEJ RELEO RELES RELOJ RELSA RELSO REMAD REMAN REMAR REMAS REMEN REMES REMOS RENAL RENDA RENDI RENES RENGA RENGO RENOS RENTA RENTE RENTO REOJO REPON REPTA REPTE REPTO RESES RESOL RESTA RESTE RESTO RETAD RETAL RETAN RETAR RETAS RETEN RETES RETIN RETOR RETOS RETRO REUMA REUNA REUNE REUNI REUNO REUSA REUSE REUSO REVEN REVES REYES REZAD REZAN REZAR REZAS REZNO REZON REZOS RIADA RIAIS RIALA RIALE RIALO RIAME RIASE RICAS RICEN RICES RICIA RICIO RICOS RIEGA RIEGO RIELA RIELE RIELO RIEME RIERA RIERE RIESE RIETE RIFAD RIFAN RIFAR RIFAS RIFEN RIFES RIFLE RIGEN RIGES RIGIO RIGOR RIJAN RIJAS RIJOS RILAD RILAN RILAR RILAS RILEN RILES RIMAD RIMAN RIMAR RIMAS RIMEL RIMEN RIMES RINDA RINDE RINDO RINGA RINGO RIOJA RIPIA RIPIE RIPIO RISAS RISCA RISCO RISOS RISPA RISPO RITAS RITMA RITME RITMO RITOS RIVAL RIZAD RIZAN RIZAR RIZAS RIZON RIZOS ROAIS ROALA ROALE ROALO ROANA ROANO ROBAD ROBAN ROBAR ROBAS ROBEN ROBES ROBIN ROBLA ROBLE ROBLO ROBOS ROBOT ROBRA ROBRE ROCAS ROCEN ROCES ROCIA ROCIE ROCIN ROCIO RODAD RODAL RODAR RODAS RODEA RODEE RODEO RODIA RODIO ROEIS ROELA ROELE ROELO ROERA ROERE ROETE ROGAD ROGAR ROGUE ROIAN ROIAS ROIDA ROIDO ROIGA ROIGO ROJAL ROJAS ROJEA ROJEE ROJEO ROJEZ ROJOS ROLAD ROLAN ROLAR ROLAS ROLDA ROLDE ROLDO ROLEN ROLEO ROLES ROLLA ROLLE ROLLO ROMAS ROMBO ROMEA ROMEO ROMOS ROMPA ROMPE ROMPI ROMPO RONCA RONCE RONCO RONDA RONDE RONDO RONES RONZA RONZO ROPAS ROPON ROQUE RORAD RORAN RORAR RORAS ROREN RORES RORRO ROSAD ROSAL ROSAN ROSAR ROSAS ROSCA ROSCO ROSEA ROSEE ROSEN ROSEO ROSES ROSJO ROSON ROSOS ROTAD ROTAL ROTAN ROTAR ROTAS ROTEN ROTES ROTOR ROTOS ROYAN ROYAS ROYOS ROZAD ROZAN ROZAR ROZAS ROZNA ROZNE ROZNO ROZON ROZOS RUABA RUADA RUADO RUAIS RUANA RUANO RUARA RUARE RUASE RUBEA RUBEO RUBIA RUBIN RUBIO RUBLO RUBOR RUBOS RUBRA RUBRO RUCAD RUCAN RUCAR RUCAS RUCHA RUCHE RUCHO RUCIA RUCIO RUCOS RUDAS RUDOS RUECA RUEDA RUEDE RUEDO RUEGA RUEGO RUEIS RUEJO RUFAS RUFON RUFOS RUGAD RUGAN RUGAR RUGAS RUGBY RUGEN RUGES RUGIA RUGID RUGIO RUGIR RUGIS RUGUE RUIDO RUINA RUINE RUINO RUJAN RUJAS RUJIA RUJIE RUJIO RULAD RULAN RULAR RULAS RULEN RULES RULOS RUMBA RUMBE RUMBO RUMIA RUMIE RUMIO RUMOR RUMOS RUNAS RUNGO RUNOS RUPIA RUQUE RURAL RUSAS RUSCO RUSIA RUSOS RUSTA RUSTE RUSTI RUSTO RUTAD RUTAN RUTAR RUTAS RUTEL RUTEN RUTES SABED SABEN SABEO SABER SABES SABIA SABIO SABLE SABOR SABRA SABRE SACAD SACAN SACAR SACAS SACES SACHA SACHE SACHO SACIA SACIE SACIO SACOS SACRA SACRE SACRO SAETA SAETI SAGAS SAGAZ SAINA SAINE SAINO SAJAD SAJAN SAJAR SAJAS SAJEN SAJES SAJIA SAJON SALAD SALAN SALAR SALAS SALAZ SALCE SALDA SALDE SALDO SALEA SALEE SALEN SALEO SALEP SALES SALGA SALGO SALIA SALID SALIN SALIO SALIR SALIS SALLA SALLE SALLO SALMA SALME SALMO SALOL SALON SALPA SALSA SALTA SALTE SALTO SALUD SALVA SALVE SALVO SAMAN SAMAS SAMBA SAMIA SAMIO SAMPA SANAD SANAN SANAR SANAS SANCO SANEA SANEE SANEN SANEO SANES SANGO SANIE SANOS SANSA SANSO SANTA SANTO SAPOS SAQUE SARAN SARAO SARCO SARDA SARDE SARDO SARGA SARGO SARIA SARNA SARRO SARTA SATAN SATEN SATIN SATIS SATOS SAUCE SAUCO SAUDI SAUNA SAVIA SAXEA SAXEO SAXOS SAYAL SAYAS SAYON SAYOS SAZON SEAIS SEBES SEBOS SECAD SECAN SECAR SECAS SECON SECOS SECTA SECUA SEDAD SEDAL SEDAN SEDAR SEDAS SEDEA SEDEE SEDEN SEDEO SEDES SEGAD SEGAR SEGAS SEGRI SEGUE SEGUI SEGUN SEGUR SEICO SEISE SELLA SELLE SELLO SELVA SEMAS SEMEN SEMIS SENAS SENDA SENES SENIL SENOS SENTE SENTI SENTO SEPAN SEPAS SEPIA SEQUE SERAN SERAS SERES SERIA SERIE SERIO SERNA SERON SERPA SERRE SERRO SERVA SERVE SERVI SERVO SESEA SESEE SESEO SESGA SESGO SESMA SESMO SESOS SETAS SETOS SEUDO SEXMO SEXOS SEXTA SEXTO SIBIL SIDAS SIDOS SIDRA SIEGA SIEGO SIENA SIESO SIETE SIFON SIGAN SIGAS SIGLA SIGLO SIGMA SIGNA SIGNE SIGNO SIGUE SILBA SILBE SILBO SILEX SILFO SILLA SILOS SILVA SIMAS SIMIA SIMIL SIMIO SIMON SINOS SIRAS SIRGA SIRGO SIRIA SIRIO SIRLE SIROS SIRTE SIRVA SIRVE SIRVO SISAD SISAN SISAR SISAS SISEA SISEE SISEN SISEO SISES SISMO SISON SITAS SITIA SITIE SITIO SITOS SITUA SITUE SITUO SOASA SOASE SOASO SOBAD SOBAN SOBAR SOBAS SOBEN SOBEO SOBES SOBON SOBOS SOBRA SOBRE SOBRO SOCAS SOCAZ SOCIA SOCIO SODAS SODIO SOEZA SOFAS SOGAS SOJAS SOLAD SOLAR SOLAS SOLAZ SOLDE SOLDO SOLEA SOLED SOLEE SOLEO SOLER SOLES SOLFA SOLIA SOLIO SOLLA SOLOS SOLTE SOLTO SOMAS SOMOS SONAD SONAR SONDA SONDE SONDO SONES SONIA SONSO SOPAD SOPAN SOPAR SOPAS SOPEA SOPEE SOPEN SOPEO SOPES SOPLA SOPLE SOPLO SOPON SOPOR SORBA SORBE SORBI SORBO SORCE SORDA SORDO SORGO SORNA SORNE SORNO SOROS SORRA SOSAR SOSAS SOSIA SOSOS SOTAS SOTES SOTOL SOTOS SOVOZ SPORT STAND SUAVE SUBAN SUBAS SUBEN SUBES SUBIA SUBID SUBIO SUBIR SUBIS SUCIA SUCIO SUCRE SUDAD SUDAN SUDAR SUDAS SUDEN SUDES SUDOR SUECA SUECO SUELA SUELE SUELO SUENA SUENE SUENO SUERO SUEVA SUEVO SUFRA SUFRE SUFRI SUFRO SUITE SUIZA SUIZO SULAS SUMAD SUMAN SUMAR SUMAS SUMEN SUMES SUMIA SUMID SUMIO SUMIR SUMIS SUMOS SUPER SUPLA SUPLE SUPLI SUPLO SUPON SUPRA SURAS SURCA SURCO SURGE SURGI SURJA SURJO SURTA SURTE SURTI SURTO SUSTO SUTAS SUTES SUTIL SUYAS SUYOS SUZON TABAS TABEA TABLA TACAD TACAN TACAR TACAS TACEN TACES TACHA TACHE TACHO TACON TACOS TACTO TAFIA TAFON TAFOS TAGUA TAHAS TAHUR TAIFA TAIGA TAIMA TAINA TAINO TAJAD TAJAN TAJAR TAJAS TAJEA TAJEN TAJES TAJON TAJOS TALAD TALAN TALAR TALAS TALCO TALEA TALEN TALES TALIN TALIO TALLA TALLE TALLO TALMA TALON TALOS TALPA TALUD TAMAL TAMBA TAMBO TAMIZ TAMOS TANDA TANGA TANGO TANKA TANOR TANTA TANTO TAPAD TAPAN TAPAR TAPAS TAPEA TAPEE TAPEN TAPEO TAPES TAPIA TAPIE TAPIN TAPIO TAPIR TAPIS TAPIZ TAPON TAQUE TARAD TARAN TARAR TARAS TARAY TARCO TARDA TARDE TARDO TAREA TAREN TARES TARIN TARJA TARJE TARJO TAROT TARRO TARSO TARTA TASAD TASAN TASAR TASAS TASCA TASCO TASEN TASES TASIA TASIO TASTO TATAS TATES TATOS TATUA TATUE TATUO TAULA TAURO TAXIS TAXON TAZAD TAZAN TAZAR TAZAS TAZON TBYTE TEAME TEBEO TECAS TECES TECHA TECHE TECHO TECLA TEDAS TEDIO TEFES TEGEA TEGEO TEGUA TEJAD TEJAN TEJAR TEJAS TEJED TEJEN TEJER TEJES TEJIA TEJIO TEJON TEJOS TELAR TELAS TELON TEMAN TEMAS TEMED TEMEN TEMER TEMES TEMIA TEMIO TEMOR TEMPO TENAS TENAZ TENDI TENED TENER TENGA TENGO TENIA TENIS TENLA TENLE TENLO TENME TENOR TENSA TENSE TENSO TENTE TENTO TENUE TEOSA TEOSO TERCA TERCO TERMA TERMO TERNA TERNE TERNO TEROS TERSA TERSE TERSO TESAD TESAN TESAR TESAS TESEN TESES TESIS TESLA TESON TESOS TESTA TESTE TESTO TETAD TETAN TETAR TETAS TETEN TETES TETON TETRA TEXAS TEXTO TEYAS TEYOS TIACA TIARA TIBES TIBIA TIBIE TIBIO TIENE TIESA TIESO TIFON TIFOS TIFUS TIGRE TIJAS TILAS TILDA TILDE TILDO TILIA TILIN TILLA TILLE TILLO TILMA TILOS TIMAD TIMAN TIMAR TIMAS TIMBA TIMBO TIMEN TIMES TIMOL TIMON TIMOS TIMPA TINAS TINCA TINGE TINOS TINTA TINTE TINTO TIPAS TIPIS TIPLE TIPOS TIQUE TIRAD TIRAN TIRAR TIRAS TIREN TIRES TIRIA TIRIO TIRON TIROS TIRSO TISIS TISTE TITAD TITAN TITAR TITAS TITEN TITES TITOS TIZAS TIZNA TIZNE TIZNO TIZON TIZOS TLACO TOABA TOADA TOADO TOAIS TOARA TOARE TOASE TOBAR TOBAS TOCAD TOCAN TOCAR TOCAS TOCEN TOCES TOCHA TOCHE TOCHO TOCIA TOCIO TOCON TOCOS TODAS TODOS TOEIS TOESA TOFOS TOGAS TOJAL TOJOS TOKEN TOLAS TOLDA TOLDE TOLDO TOLES TOLLA TOLLO TOLMO TOLON TOLVA TOMAD TOMAN TOMAR TOMAS TOMEN TOMES TOMON TOMOS TONAD TONAL TONAN TONAR TONAS TONCA TONDO TONEL TONEN TONER TONES TONGA TONGO TONOS TONTA TONTO TOPAD TOPAN TOPAR TOPAS TOPEN TOPES TOPOS TOQUE TORAL TORAS TORAX TORCA TORCE TORCI TORCO TORDA TORDO TOREA TOREE TOREO TORGA TORGO TORIL TORIO TORMO TORNA TORNE TORNO TOROS TORPE TORRA TORRE TORRO TORSO TORTA TORVA TORVO TOSAN TOSAS TOSCA TOSCO TOSED TOSEN TOSER TOSES TOSIA TOSIO TOSTE TOSTO TOTAL TOTEM TOVAS TOZAD TOZAN TOZAR TOZAS TOZOS TRABA TRABE TRABO TRACA TRACE TRAED TRAEN TRAER TRAES TRAGA TRAGO TRAIA TRAJE TRAJO TRAMA TRAME TRAMO TRAPA TRAPE TRAPO TRAPS TRARO TRATA TRATE TRATO TRAZA TRAZO TREBO TRECE TREJA TREMA TREME TREMI TREMO TRENA TRENO TREOS TREPA TREPE TREPO TRETA TRIAD TRIAL TRIAN TRIAR TRIAS TRIBU TRICE TRIEN TRIES TRIGA TRIGO TRILE TRINA TRINE TRINO TRIOS TRIPA TRIPE TRISA TRIZA TRIZO TROCE TROCO TROJE TROLA TROLE TRONA TRONE TRONO TROPA TROPO TROTA TROTE TROTO TROVA TROVE TROVO TROYA TROZA TROZO TRUCA TRUCO TRUFA TRUFE TRUFO TRUJA TRUSA TRUST TUBAS TUBOS TUCAN TUCAS TUCOS TUDAS TUECA TUECO TUERA TUFOS TULES TULIO TULLA TULLE TULLI TULLO TUMBA TUMBE TUMBO TUMOR TUMOS TUNAD TUNAN TUNAR TUNAS TUNCO TUNDA TUNDE TUNDI TUNDO TUNEA TUNEE TUNEL TUNEN TUNEO TUNES TUNEZ TUNOS TUPAN TUPAS TUPEN TUPES TUPIA TUPID TUPIN TUPIO TUPIR TUPIS TUPLA TURBA TURBE TURBO TURCA TURCO TURMA TURNA TURNE TURNO TURON TURRA TURRE TURRO TUSAS TUSON TUTEA TUTEE TUTEO TUTES TUTOR TUYAS TUYOS UBICA UBICO UBIOS UBRES UCASE UFANA UFANE UFANO UGRES UJIER ULEMA ULTRA ULULA ULULE ULULO UMERO UNAIS UNALA UNALE UNALO UNAME UNASE UNCEN UNCES UNCIA UNCID UNCIO UNCIR UNCIS UNELA UNELE UNELO UNEME UNETE UNGEN UNGES UNGIA UNGID UNGIO UNGIR UNGIS UNIAN UNIAS UNICA UNICO UNIDA UNIDO UNION UNIOS UNIRA UNIRE UNJAN UNJAS UNTAD UNTAN UNTAR UNTAS UNTEN UNTES UNTOS UNZAN UNZAS URANO URAPE URATO URBES URDAN URDAS URDEN URDES URDIA URDID URDIO URDIR URDIS UREAS URGEN URGES URGIA URGID URGIO URGIR URGIS URICA URICO URJAN URJAS URNAS URUBU URUCU URUGA USABA USADA USADO USAIS USAJE USALA USALE USALO USAME USAOS USARA USARE USART USASE USATE USEIS USELA USELE USELO USEME USESE USIAS USTED USUAL USURA USURE USURO UTERO UVADA UVEAS UVERA UVERO UVULA VACAD VACAN VACAR VACAS VACIA VACIE VACIO VACOS VACUA VACUO VADEA VADEE VADEO VADES VADOS VAGAD VAGAN VAGAR VAGAS VAGON VAGOS VAGUE VAHOS VAIDA VAINA VALAR VALED VALEN VALER VALES VALGA VALGO VALIA VALIO VALLA VALLE VALLO VALON VALOR VALSA VALSE VALSO VALUA VALUE VALUO VALVA VAMOS VANAS VANEA VANEE VANEO VANOS VAPOR VAQUE VARAD VARAL VARAN VARAR VARAS VAREA VAREE VAREN VAREO VARES VARGA VARIA VARIE VARIO VARIZ VARON VASAR VASCA VASCO VASOS VASTA VASTO VATER VATES VATIO VAYAN VAYAS VEAIS VEALA VEALE VEALO VEAME VEASE VECEN VECES VEDAD VEDAN VEDAR VEDAS VEDEN VEDES VEDLA VEDLE VEDLO VEDME VEGAS VEIAN VEIAS VEJAD VEJAN VEJAR VEJAS VEJEN VEJES VEJEZ VELAD VELAN VELAR VELAS VELEN VELES VELLO VELON VELOS VELOZ VEMOS VENAL VENAS VENCE VENCI VENDA VENDE VENDI VENDO VENGA VENGO VENIA VENID VENIR VENIS VENME VENOS VENTA VENTE VENTO VENUS VENZA VENZO VERAN VERAS VERAZ VERBO VERDE VERGA VERGE VERIA VERIL VERJA VERLA VERLE VERLO VERME VERMU VEROS VERSA VERSE VERSO VERTE VERTI VESTE VESTI VETAD VETAN VETAR VETAS VETEA VETEE VETEN VETEO VETES VETOS VEZAD VEZAN VEZAR VEZAS VIADA VIAJA VIAJE VIAJO VIBRA VIBRE VIBRO VICES VICIA VICIE VICIO VICTO VIDAS VIDEO VIDES VIEJA VIEJO VIENE VIERA VIERE VIESE VIGAS VIGIA VIGIE VIGIO VIGOR VILES VILLA VILOS VIMOS VINAR VINCO VINOS VINTA VIOLA VIOLE VIOLO VIRAD VIRAL VIRAN VIRAR VIRAS VIREN VIRES VIRGO VIRIL VIRON VIRUS VISAD VISAN VISAR VISAS VISCO VISEA VISEE VISEN VISEO VISES VISIR VISON VISOR VISOS VISTA VISTE VISTO VITAE VITAL VITOR VITOS VITRE VIUDA VIUDO VIVAC VIVAN VIVAR VIVAS VIVAZ VIVEN VIVES VIVIA VIVID VIVIO VIVIR VIVIS VIVOS VOCAL VOCEA VOCEE VOCEO VOCES VODKA VOLAD VOLAR VOLCO VOLEA VOLEE VOLEO VOLVI VORAZ VOSEA VOSEE VOSEO VOTAD VOTAN VOTAR VOTAS VOTEN VOTES VOTOS VUELA VUELE VUELO VULGO VULVA WEBER YABAS YACED YACEN YACER YACES YACIA YACIO YAGAN YAGAS YAMBO YANAS YAQUI YARDA YATES YAYAS YAYOS YAZCA YAZCO YAZGA YAZGO YEBOS YECOS YEDRA YEGUA YELMO YEMAS YENDO YENES YERBA YERMA YERME YERMO YERNO YEROS YERRA YERRE YERRO YERTA YERTO YERVO YESAR YESCA YESON YESOS YETIS YEZGO YODOS YOGAS YOGUR YOYOS YUGOS YUNTA YUNTO YUTES ZAFAD ZAFAN ZAFAR ZAFAS ZAFEN ZAFES ZAFIA ZAFIO ZAFOS ZAFRA ZAGAL ZAGAS ZAHON ZAINA ZAINO ZAMBO ZAMPA ZAMPE ZAMPO ZANCA ZANCO ZANGA ZANJA ZANJE ZANJO ZAPAD ZAPAN ZAPAR ZAPAS ZAPEA ZAPEE ZAPEN ZAPEO ZAPES ZAQUE ZARBO ZARCA ZARCO ZARES ZARJA ZARPA ZARPE ZARPO ZARZA ZARZO ZATAS ZAYAS ZEDAS ZENDA ZENDO ZENIT ZETAS ZILOG ZOCAS ZOCOS ZOFRA ZOIZO ZOMAS ZOMPA ZOMPO ZONAL ZONAS ZOPOS ZORRA ZORRO ZOTES ZUAVO ZUBIA ZUDAS ZUECO ZUELA ZUIZA ZULOS ZUMBA ZUMBE ZUMBO ZUMOS ZURCE ZURCI ZURDA ZURDO ZUREA ZUREE ZUREO ZUROS ZURRA ZURRE ZURRI ZURRO ZURZA ZURZO".split(" ")

combinations = ["00000", "00001", "00002", "00010", "00011", "00012", "00020", "00021", "00022", "00100", "00101", "00102", "00110", "00111", "00112", "00120", "00121", "00122", "00200", "00201", "00202", "00210", "00211", "00212", "00220", "00221", "00222", "01000", "01001", "01002", "01010", "01011", "01012", "01020", "01021", "01022", "01100", "01101", "01102", "01110", "01111", "01112", "01120", "01121", "01122", "01200", "01201", "01202", "01210", "01211", "01212", "01220", "01221", "01222", "02000", "02001", "02002", "02010", "02011", "02012", "02020", "02021", "02022", "02100", "02101", "02102", "02110", "02111", "02112", "02120", "02121", "02122", "02200", "02201", "02202", "02210", "02211", "02212", "02220", "02221", "02222", "10000", "10001", "10002", "10010", "10011", "10012", "10020", "10021", "10022", "10100", "10101", "10102", "10110", "10111", "10112", "10120", "10121", "10122", "10200", "10201", "10202", "10210", "10211", "10212", "10220", "10221", "10222", "11000", "11001", "11002", "11010", "11011", "11012", "11020", "11021", "11022", "11100", "11101", "11102", "11110", "11111", "11112", "11120", "11121", "11122", "11200", "11201", "11202", "11210", "11211", "11212", "11220", "11221", "11222", "12000", "12001", "12002", "12010", "12011", "12012", "12020", "12021", "12022", "12100", "12101", "12102", "12110", "12111", "12112", "12120", "12121", "12122", "12200", "12201", "12202", "12210", "12211", "12212", "12220", "12221", "12222", "20000", "20001", "20002", "20010", "20011", "20012", "20020", "20021", "20022", "20100", "20101", "20102", "20110", "20111", "20112", "20120", "20121", "20122", "20200", "20201", "20202", "20210", "20211", "20212", "20220", "20221", "20222", "21000", "21001", "21002", "21010", "21011", "21012", "21020", "21021", "21022", "21100", "21101", "21102", "21110", "21111", "21112", "21120", "21121", "21122", "21200", "21201", "21202", "21210", "21211", "21212", "21220", "21221", "21222", "22000", "22001", "22002", "22010", "22011", "22012", "22020", "22021", "22022", "22100", "22101", "22102", "22110", "22111", "22112", "22120", "22121", "22122", "22200", "22201", "22202", "22210", "22211", "22212", "22220", "22221", "22222"]


def generatePattern(entrada, word):
    pattern = ""
    procesed = {}

    for j in range(len(entrada)):
        letra = word[j]
        if letra not in procesed:
            condition = max([word[k] == letra and entrada[k] == 2 for k in range(j + 1, len(entrada))] + [False])
            if entrada[j] == 0:
                if not condition: procesed[letra] = 0
            else:
                procesed[letra] = 0
            pattern += [f"(?=[^{letra}]*$)" if not condition else f"(?!.{{{j}}}{letra})", f"(?!.{{{j}}}{letra})(?=.*{letra})",
                 f"(?=.{{{j}}}{letra})"][
                    entrada[j]]
    return f"^{pattern}.*$"

d = {}
for i in s:
    print(i)
    d[i] = {}
    for c in combinations:
        d[i][c] = generatePattern([int(a) for a in c], i)
print(len(s), len(d), len(d["ABACO"]), d)
with open('file2.txt', 'w') as file:
     file.write(json.dumps(d))